# MFA-Modell for Timber Trusses in Eichkampkieyz


* **Version:** 1.0
* **Date:** 14.09.2025
* **Authors:** [A. K., Johannes Scholz, Lukas Hoppe]



# Section 0: Importing the packages & basic settings translator

In [1]:
### Load packages ###
# Add this at the beginning of your notebook
!jupyter nbextension list

# Load general libraries
import sys, os
import numpy as np
import pandas as pd
from scipy.stats import lognorm
import xlsxwriter
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
import re
from collections import defaultdict
from scipy.optimize import minimize
import copy
from ipywidgets import SelectMultiple
import networkx as nx
from matplotlib.patches import Rectangle
import warnings
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, Dropdown


# Load ODYM package
# Add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'framework', 'ODYM-master_20241127', 'odym', 'modules')) 

# Import the ODYM class file
import ODYM_Classes as msc 
# Import the ODYM function file
import ODYM_Functions as msf
# Import the dynamic stock model library
import dynamic_stock_model as dsm 


# Load bioDYM_addon
# Add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'framework', 'bioDYM_add-on', 'modules')) 

# Import classes for first order model process
import bioDYM_classes as bicl
# Import plotting functions
import bioDYM_plotting as bipl
# Import export functions
import bioDYM_export as bix



# Enables plotting directly in the notebook (in most cases this is already enabled)
%matplotlib inline

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console contrib dejavu events execute kernel kernelspec
lab labextension labhub migrate nbconvert nbextensions_configurator notebook
qtconsole run script server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [2]:
pd.DataFrame()

""


# Section 1: Configuration

In [3]:
# ====================================================================
# Section 1: Model Configuration
# All user-defined settings for a scenario run go here.
# ====================================================================

# --- File Paths ---
EXCEL_FILE_PATH = '250731_V11.xlsx'

# --- Model Scope ---
START_YEAR = 1920
END_YEAR = 2045
ELEMENTS = ['material', 'WC', 'DM', 'CC'] # Elements to be tracked

# In Section 1: Model Configuration

# --- Model Calculation Switches ---
# Set to True to enable the calculation, False to disable.
# This allows for simpler test runs without DSM or FOMP.
RUN_DSM_CALCULATION = True
RUN_FOMP_CALCULATION = False # Set to False to disable FOMP calculations

# --- Dynamic Stock Model (DSM) Parameters ---
# This dictionary holds the parameters for ALL dynamic stocks in the model.
# The key (e.g., 3) is the integer process_id where the stock is located.
# This structure allows for multiple DSMs; just add another process_id as a key.
# In Section 1: Model Configuration
DSM_PARAMS = {
    0: { # Parameters for Stock in process 0: Biosphere #
        'inflow_split': [1], # Split of inflow into the stock
        'lifetimes': {
            'Type': 'Normal', # Type of distribution for residence time
            'Mean': [20], # Residence time in process
            'StdDev': [4] # Standard deviation for the residence time
        },
        # Name for inflow categories in the legend of the plot
        'category_names': ['Average Exchange'] 
    },

    2: { # Parameters for Stock in process 2: Processing #
        'inflow_split': [1], 
        'lifetimes': {
            'Type': 'Normal', 
            'Mean': [6],
            'StdDev': [2]
        },
        'category_names': ['Air Drying'] 
    },

    3: { # Parameters for Stock in process 3: Use #
        'inflow_split': [1], 
        'lifetimes': {
            'Type': 'Normal',
            'Mean': [80],
            'StdDev': [16] 
        },
    'category_names': ['Average Lifetime'] 
    },


    7: { # Parameters for Stock in process 7: Atmosphere #
        'inflow_split': [1], 
        'lifetimes': {
            'Type': 'Normal',
            'Mean': [70], 
            'StdDev': [2]
        },
     'category_names': ['Photosynthesis']  
    },

    8: { # Parameters for Stock in process 8: Anthroposphere #
        'inflow_split': [1],
        'lifetimes': {
            'Type': 'Normal',
            'Mean': [], 
            'StdDev': []
        },
    'category_names': [] 
    }
}
# Example for a second dynamic stock in another process (e.g., process 25):
# 25: {
#     'lifetimes': {
#         'Type': 'Lognormal',
#         'Mean': [50],
#         'StdDev': [15]
#     }
# }

# --- First-Order Model Process (FOMP) Parameters ---
# This dictionary holds parameters for ALL FOMP calculations.
# The key (e.g., 17) is the integer process_id where the decay occurs.
# In Section 1: Model Configuration

# FOMP_PARAMS = {
#     5: { # Parameter für die Mineralisierung im Prozess 17 (Lithosphere_Stock)
#         'outflow_id': 'F_05_07', # NEU: Der exakte Name des Abflusses, der berechnet werden soll
#         'f': 0.5, 
#         'k1': 0.5, 
#         'k2': 0.5
#     }
# }

    
    # Example for a second FOMP in another process (e.g., process 16):
    # 16: {
    #     'f': 0.1,
    #     'k1': 0.05,
    #     'k2': 0.001
    # }

In [4]:
# Enhanced Validation for DSM Lifetimes
# This function checks and corrects DSM lifetime parameters to ensure realistic values.
def validate_dsm_lifetimes(dsm_params):
    """Enhanced validation for DSM lifetimes
    Ensures lifetimes and standard deviations are within realistic bounds.
    """
    for process_id, params in dsm_params.items():
        lifetimes = params['lifetimes']['Mean']
        std_devs = params['lifetimes']['StdDev']
        
        for i, (lt, std) in enumerate(zip(lifetimes, std_devs)):
            # Check minimum lifetime
            if lt < 2.0:  # Minimum 2 years for processing
                print(f"⚠️ WARNING: Process {process_id} lifetime {lt} years too short")
                print(f"   Minimum recommended: 2 years for processing stability")
                params['lifetimes']['Mean'][i] = 2.0
                print(f"   Auto-corrected to {params['lifetimes']['Mean'][i]} years")
            
            # Check std dev vs mean ratio
            if std > lt * 0.8:  # StdDev shouldn't exceed 80% of mean
                print(f"⚠️ WARNING: Process {process_id} StdDev {std} too large vs Mean {lt}")
                params['lifetimes']['StdDev'][i] = lt * 0.5  # Set to 50% of mean
                print(f"   Auto-corrected StdDev to {params['lifetimes']['StdDev'][i]} years")
                
    return dsm_params


# Apply enhanced validation
DSM_PARAMS = validate_dsm_lifetimes(DSM_PARAMS)

In [5]:
print(DSM_PARAMS)
#print(FOMP_PARAMS)

{0: {'inflow_split': [1], 'lifetimes': {'Type': 'Normal', 'Mean': [20], 'StdDev': [4]}, 'category_names': ['Average Exchange']}, 2: {'inflow_split': [1], 'lifetimes': {'Type': 'Normal', 'Mean': [6], 'StdDev': [2]}, 'category_names': ['Air Drying']}, 3: {'inflow_split': [1], 'lifetimes': {'Type': 'Normal', 'Mean': [80], 'StdDev': [16]}, 'category_names': ['Average Lifetime']}, 7: {'inflow_split': [1], 'lifetimes': {'Type': 'Normal', 'Mean': [70], 'StdDev': [2]}, 'category_names': ['Photosynthesis']}, 8: {'inflow_split': [1], 'lifetimes': {'Type': 'Normal', 'Mean': [], 'StdDev': []}, 'category_names': []}}


# Section 2: Function definitions

In [6]:
# ====================================================================
# Section 2: Function Definitions
# ====================================================================

# --- Helper Function 2.1: Define Model Scope & Classifications ---


def define_model_scope(start_year, end_year, elements):
    """
    Defines the temporal and elemental scope of the MFA model. 
    Creates the ModelClassification dictionary and IndexTable DataFrame,
    which are core ODYM objects used for calculations

    Args:
        start_year (int): The first year of the analysis.
        end_year (int): The last year of the analysis.
        elements (list): A list of strings for the elements to be tracked.

    Returns:
        tuple: A tuple containing the ModelClassification dictionary 
               and the IndexTable DataFrame.
    """
    ModelClassification = {} # Initialize an empty dictionary to hold model classifications
    MyYears = list(np.arange(start_year, end_year + 1)) # Generata a list of years (MyYears) from start_year to end_year (inclusive)

    
    # Define Time and Element Classifications for the ODYM framework
    # Creates a dictionary called ModelClassification that stores ODYM Classification objects for 'Time' and 'Element'.
    ModelClassification['Time'] = msc.Classification(Name='Time', Dimension='Time', ID=1, Items=MyYears)
    ModelClassification['Element'] = msc.Classification(Name='Elements', Dimension='Element', ID=2, Items=elements)

    # Create the IndexTable (Pandas DataFrame), which ODYM uses for calculations
    IndexTable = pd.DataFrame({
        'Aspect': ['Time', 'Element'],
        'Description': ['Model aspect "time"', 'Model aspect "Element"'],
        'Dimension': ['Time', 'Element'],
        'Classification': [ModelClassification[Aspect] for Aspect in ['Time', 'Element']],
        'IndexLetter': ['t', 'e']
    })
    IndexTable.set_index('Aspect', inplace=True)
    
    print("--> Model scope and classifications defined.")
    return ModelClassification, IndexTable

In [7]:
# --- Test Function for define_model_scope ---
def test_define_model_scope():
    """
    Test function to verify that define_model_scope works correctly.
    Tests various scenarios and validates the output structure.
    """
    print("=== Testing define_model_scope Function ===\n")
    
    try:
        # Test 1: Basic functionality with your project parameters
        print("Test 1: Basic functionality")
        start_year = 1920
        end_year = 2045
        elements = ['material', 'WC', 'DM', 'CC']
        
        model_classification, index_table = define_model_scope(start_year, end_year, elements)
        
        # Validate results
        assert isinstance(model_classification, dict), "ModelClassification should be a dictionary"
        assert 'Time' in model_classification, "Time classification missing"
        assert 'Element' in model_classification, "Element classification missing"
        
        # Check time classification
        time_items = model_classification['Time'].Items
        assert len(time_items) == (end_year - start_year + 1), f"Expected {end_year - start_year + 1} years, got {len(time_items)}"
        assert time_items[0] == start_year, f"First year should be {start_year}, got {time_items[0]}"
        assert time_items[-1] == end_year, f"Last year should be {end_year}, got {time_items[-1]}"
        
        # Check element classification
        element_items = model_classification['Element'].Items
        assert element_items == elements, f"Elements don't match. Expected {elements}, got {element_items}"
        
        # Check IndexTable structure
        assert isinstance(index_table, pd.DataFrame), "IndexTable should be a DataFrame"
        assert list(index_table.index) == ['Time', 'Element'], "IndexTable index should be ['Time', 'Element']"
        
        print("✅ Test 1 passed: Basic functionality works correctly")
        print(f"   - Years: {start_year} to {end_year} ({len(time_items)} years)")
        print(f"   - Elements: {elements}")
        print(f"   - IndexTable shape: {index_table.shape}")
        
    except Exception as e:
        print(f"❌ Test 1 failed: {e}")
        return False
    
    try:
        # Test 2: Edge case - single year
        print("\nTest 2: Edge case - single year")
        model_classification_2, index_table_2 = define_model_scope(2023, 2023, ['carbon'])
        
        time_items_2 = model_classification_2['Time'].Items
        assert len(time_items_2) == 1, f"Expected 1 year, got {len(time_items_2)}"
        assert time_items_2[0] == 2023, f"Expected 2023, got {time_items_2[0]}"
        
        print("✅ Test 2 passed: Single year case works correctly")
        
    except Exception as e:
        print(f"❌ Test 2 failed: {e}")
        return False
    
    try:
        # Test 3: Different element set
        print("\nTest 3: Different element configuration")
        model_classification_3, index_table_3 = define_model_scope(2000, 2010, ['mass', 'carbon', 'nitrogen'])
        
        element_items_3 = model_classification_3['Element'].Items
        expected_elements = ['mass', 'carbon', 'nitrogen']
        assert element_items_3 == expected_elements, f"Expected {expected_elements}, got {element_items_3}"
        
        print("✅ Test 3 passed: Different element set works correctly")
        
    except Exception as e:
        print(f"❌ Test 3 failed: {e}")
        return False
    
    # Test 4: Display detailed information about the objects created
    print("\nTest 4: Detailed object inspection")
    try:
        model_classification, index_table = define_model_scope(1920, 2045, ['material', 'WC', 'DM', 'CC'])
        
        print("📊 ModelClassification structure:")
        for key, classification in model_classification.items():
            print(f"   {key}: {classification.__class__.__name__}")
            print(f"      - Name: {classification.Name}")
            print(f"      - Dimension: {classification.Dimension}")
            print(f"      - ID: {classification.ID}")
            print(f"      - Items count: {len(classification.Items)}")
            if key == 'Time':
                print(f"      - First/Last: {classification.Items[0]}/{classification.Items[-1]}")
            else:
                print(f"      - Items: {classification.Items}")
        
        print("\n📊 IndexTable structure:")
        print(index_table)
        print(f"   Index: {list(index_table.index)}")
        print(f"   Columns: {list(index_table.columns)}")
        
        print("✅ Test 4 passed: Detailed inspection completed")
        
    except Exception as e:
        print(f"❌ Test 4 failed: {e}")
        return False
    
    print("\n🎉 All tests passed! The define_model_scope function is working correctly.")
    return True


# === Main execution block ===
if __name__ == "__main__":
    # Run the test function
    test_result = test_define_model_scope()
    
    if test_result:
        print("\n" + "="*50)
        print("READY TO USE: define_model_scope function is validated!")
        print("You can now use it in your MFA workflow.")
        print("="*50)
    else:
        print("\n" + "="*50)
        print("ISSUES FOUND: Please review the function implementation.")
        print("="*50)


=== Testing define_model_scope Function ===

Test 1: Basic functionality
--> Model scope and classifications defined.
✅ Test 1 passed: Basic functionality works correctly
   - Years: 1920 to 2045 (126 years)
   - Elements: ['material', 'WC', 'DM', 'CC']
   - IndexTable shape: (2, 4)

Test 2: Edge case - single year
--> Model scope and classifications defined.
✅ Test 2 passed: Single year case works correctly

Test 3: Different element configuration
--> Model scope and classifications defined.
✅ Test 3 passed: Different element set works correctly

Test 4: Detailed object inspection
--> Model scope and classifications defined.
📊 ModelClassification structure:
   Time: Classification
      - Name: Time
      - Dimension: Time
      - ID: 1
      - Items count: 126
      - First/Last: 1920/2045
   Element: Classification
      - Name: Elements
      - Dimension: Element
      - ID: 2
      - Items count: 4
      - Items: ['material', 'WC', 'DM', 'CC']

📊 IndexTable structure:
            

In [8]:
# --- Helper Function 2.2: Initialize the main MFA System object ---

def initialize_mfa_system(model_classification, index_table):
    """
    Initializes the main MFAsystem object based on the defined scope.

    Args:
        model_classification (dict): The ModelClassification dictionary from define_model_scope.
        index_table (pd.DataFrame): The IndexTable DataFrame from define_model_scope.

    Returns:
        odym.MFAsystem (MFAsystem object): An empty but structured MFAsystem object.
    """
    # Extract scope details from the input objects
    start_time = model_classification['Time'].Items[0]
    end_time = model_classification['Time'].Items[-1]
    element_items = model_classification['Element'].Items
    
    # Create the main system object using the ODYM class
    MFA_System = msc.MFAsystem(
        Name='TimberTruss_MFA', 
        Geogr_Scope='Eichkamp_Kiez', 
        Unit='Mg', 
        ProcessList=[], 
        FlowDict={}, 
        StockDict={},
        ParameterDict={}, 
        Time_Start=start_time, 
        Time_End=end_time, 
        IndexTable=index_table, 
        Elements=element_items
    )
    
    print("--> MFA system object initialized.")
    return MFA_System


In [9]:
# Helper Function 2.3: Load data and define processes and stocks (Updated with Fixed Tracking)

def load_and_define_processes(mfa_system, EXCEL_FILE_PATH):
    """
    Loads all data from the Excel file, treating 'N.A.' as a missing value,
    and populates the ProcessList and StockDict in the MFA system.
    Prints how many processes, stocks, transfer coefficients, and dynamic transfer coefficients were created.

    Args: 
        mfa_system (msc.MFAsystem): The initialized MFA system object from initialize_mfa_system.
        EXCEL_FILE_PATH (str): Path to the Excel file containing process definitions.
    Returns:
        tuple: A tuple containing the updated MFA system object and the input data as a DataFrame.
    """

    warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

    # Load Excel file
    input_data = pd.read_excel(
        EXCEL_FILE_PATH,
        sheet_name=None,
        header=0,
        engine='openpyxl',
        na_values=['N.A.', 'NA', 'n/a']
    )
    print(f"--> Excel file '{EXCEL_FILE_PATH}' loaded successfully.")

    process_definitions = input_data.get('2_1_Definition_Processes')
    if process_definitions is None:
        raise ValueError("Sheet '2_1_Definition_Processes' not found in Excel file.")

    # Initialize tracking variables
    created_processes = []
    created_stocks = []
    tc_count = 0
    dynamic_tc_count = 0
    
    # Track different types of processes
    tc_processes = []
    dynamic_tc_processes = []
    stock_processes = []

    print("\n=== PROCESSING PROCESS DEFINITIONS ===")
    
    for index, row in process_definitions.iterrows():
        if pd.notna(row['Name(EN)']): # Check if the process name is not empty
            process_id = int(row['ID']) # Convert ID to integer
            process_name = row['Name(EN)'] # Process name 
            
            # Parse process characteristics 
            has_tc = str(row.get('TC?', '')).strip().lower() in ['yes', 'y', 'true', '1'] # Check if TC is defined
            has_dynamic_tc = str(row.get('Dyn_TC?', '')).strip().lower() in ['yes', 'y', 'true', '1'] # Check if dynamic TC is defined
            has_stock = str(row.get('Stock?', '')).strip().lower() in ['yes', 'y', 'true', '1'] # Check if stock is defined
            
            print(f"  Processing: {process_name} (ID: {process_id})")
            print(f"    TC: {has_tc}, Dynamic TC: {has_dynamic_tc}, Stock: {has_stock}")

            # Add process to system
            extensions = [] 
            if has_tc: 
                extensions.append('TC')
            if has_dynamic_tc:
                extensions.append('DynTC')
            if has_stock:
                extensions.append('Stock')
            
            extension_str = ','.join(extensions) if extensions else 'None' 
            # Create the process object and append it to the system
            mfa_system.ProcessList.append(
                msc.Process(Name=process_name, ID=process_id, Extensions=extension_str)
            )
            created_processes.append(process_name)

            # Track process types
            if has_tc:
                tc_processes.append(process_name)
                tc_count += 1
                
            if has_dynamic_tc:
                dynamic_tc_processes.append(process_name)
                dynamic_tc_count += 1
                
            if has_stock:
                stock_processes.append(process_name)

            # Add stocks if specified
            if has_stock:
                ds_name = f"dS_{process_id}" # Dynamic stock name
                s_name = f"S_{process_id}" # Static stock name
                
                mfa_system.StockDict[ds_name] = msc.Stock(
                    Name=ds_name, P_Res=process_id, Type=1, Indices='t,e', Values=None
                )
                mfa_system.StockDict[s_name] = msc.Stock(
                    Name=s_name, P_Res=process_id, Type=0, Indices='t,e', Values=None
                )
                
                created_stocks.extend([ds_name, s_name])
                print(f"    ✅ Created stocks: {ds_name}, {s_name}")

    # Initialize stock values
    print("\n--> Initializing stock values...")
    mfa_system.Initialize_StockValues() 

    # Enhanced summary output
    print("\n" + "="*50)
    print("PROCESS LOADING SUMMARY")
    print("="*50)
    
    print(f"📊 PROCESSES:")
    print(f"  Total processes created: {len(created_processes)}")
    print(f"  Process names: {created_processes}")
    
    print(f"\n📈 TRANSFER COEFFICIENTS:")
    print(f"  Static TC processes: {tc_count}")
    if tc_processes:
        print(f"  TC process names: {tc_processes}")
    
    print(f"  Dynamic TC processes: {dynamic_tc_count}")
    if dynamic_tc_processes:
        print(f"  Dynamic TC process names: {dynamic_tc_processes}")
    
    print(f"\n📦 STOCKS:")
    print(f"  Total stocks created: {len(created_stocks)}")
    if stock_processes:
        print(f"  Stock process names: {stock_processes}")
        print(f"  Stock names: {created_stocks}")
    
    if created_stocks:
        try:
            example_stock = mfa_system.StockDict[created_stocks[0]]
            if hasattr(example_stock, 'Values') and example_stock.Values is not None:
                print(f"  Example stock array shape: {example_stock.Values.shape}")
            else:
                print("  Stock arrays initialized but not yet populated")
        except Exception as e:
            print(f"  Could not determine stock array shape: {e}")
    else:
        print("  No stocks were created.")
    
    print("\n" + "="*50)
    print("PROCESS LOADING COMPLETE")
    print("="*50)

    # Validation checks
    print(f"\n🔍 VALIDATION CHECKS:")
    print(f"  ✅ Processes in system: {len(mfa_system.ProcessList)}")
    print(f"  ✅ Stocks in system: {len(mfa_system.StockDict)}")
    
    # Check for potential issues
    if tc_count == 0 and dynamic_tc_count == 0:
        print("  ⚠️  WARNING: No transfer coefficients defined")
    
    if len(created_stocks) == 0:
        print("  ⚠️  WARNING: No stocks defined")
    
    if len(created_processes) == 0:
        print("  ❌ ERROR: No processes were created!")
        
    print()
    return mfa_system, input_data

In [10]:
# Helper Function 2.4: Create dynamic TC time series from long table with interpolation

def create_dynamic_tc_parameters(dynamic_tc_data, time_vector):
    """
    Generates time series for TCs by reading a long data table and interpolating
    between the defined data points for each year in the model scope.

    Args:
        dynamic_tc_data (pd.DataFrame): DataFrame loaded from the dynamic TCs sheet.
        time_vector (list): The list of years for the analysis.

    Returns:
        dynamic_tc_dict (dict): A dictionary of TC names and their complete time series arrays.
    """
    print("--> Generating dynamic TC time series via interpolation...")
    dynamic_tc_dict = {}
    
    if 'TC_ID' not in dynamic_tc_data.columns:
        print("Warning: 'TC_ID' column not found in dynamic TCs sheet. Skipping.")
        return {}
        
    unique_tc_ids = dynamic_tc_data['TC_ID'].dropna().unique() # Ensure unique TC_IDs and drop NaN values
    
    # Check if there are any unique TC_IDs
    for tc_id in unique_tc_ids:
        tc_points = dynamic_tc_data[dynamic_tc_data['TC_ID'] == tc_id] 

        # Ensure there are 'Year' and 'Value' columns
        if not {'Year', 'Value'}.issubset(tc_points.columns):
            print(f"Skipping TC_ID {tc_id} due to missing columns.")
            continue

        ts = pd.Series(tc_points['Value'].values, index=tc_points['Year']) # Create a time series with 'Year' as index
        ts_full = ts.reindex(time_vector) # Reindex to the full time vector
        ts_interpolated = ts_full.interpolate(method='linear', limit_direction='both') # Interpolate missing values with a linear method
        dynamic_tc_dict[tc_id] = ts_interpolated.to_numpy() # Convert to numpy array for consistency

    print(f"Generated {len(dynamic_tc_dict)} dynamic TC parameter(s).")
    return dynamic_tc_dict


# === Main Execution Block ===
if __name__ == "__main__":
    # Define model years
    time_vector = list(range(1920, 2046)) # From 1920 to 2045 inclusive

    # Excel file details
    sheet_name = "2_5_dynamic_tcs"

    try:
        # Load the Excel sheet into a DataFrame
        dynamic_tc_data = pd.read_excel(EXCEL_FILE_PATH, sheet_name=sheet_name, engine='openpyxl')
        print("✅ Excel data loaded successfully. Preview:")
        print(dynamic_tc_data.head())

        # Run the function
        result = create_dynamic_tc_parameters(dynamic_tc_data, time_vector)

        # Print each result
        for tc_id, series in result.items():
            print(f"\nTC_ID: {tc_id}")
            print(series)

    except Exception as e:
        print(f"Error reading Excel file or processing data: {e}")

✅ Excel data loaded successfully. Preview:
      -  ID  Process_ID      Name(EN)     Output_Flow  TC? DynTC?  Flow_ID  \
0  True   1           7    Atmosphere         Atm_Bio  Yes     No  F_07_00   
1  True   2           0     Biosphere            Wood  Yes     No  F_00_01   
2  True   3           5   Degredation         Deg_Bio  Yes     No  F_05_00   
3  True   4           5   Degredation  Deg_Atm_Carbon  Yes     No  F_05_07   
4  True   5           6  Incineration  Inc_Carbon_Atm  Yes     No  F_06_07   

      TC_ID  Year  ...  Flow_ID_O_6 TC_ID_O_6_Year  TC_ID_O_6 TC O_6_[%]  \
0  TC_07_00  1920  ...          NaN        F_07_00        NaN        NaN   
1  TC_00_01  1920  ...          NaN        F_00_01        NaN        NaN   
2  TC_05_00  1920  ...          NaN        F_05_00        NaN        NaN   
3  TC_05_07  1920  ...          NaN        F_05_07        NaN        NaN   
4  TC_06_07  1920  ...          NaN        F_06_07        NaN        NaN   

   Sum_TC_O  Titel Year publica

c:\Users\nasty\anaconda3\envs\bioDYM_anaconda_environment\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
# Helper Function 2.5: Define flows and all model parameters 

def define_flows_and_parameters(mfa_system, all_excel_data, dsm_params_config):
    """
    Defines all flows and parameters, with ONLY F_00_01 getting data from Excel.
    This function sets up the flow structure, adds numerical data to flows,
    defines all parameters, and calculates elemental compositions for primary input flows.

    Args:
        mfa_system (msc.MFAsystem): The initialized MFA system object from initialize_mfa_system from load_and_define_processes
        all_excel_data (dict): Dictionary containing all loaded Excel data from load_and_define_processes.
        dsm_params_config (dict): Configuration dictionary for DSM parameters.

    Returns:
        tuple: A tuple containing the updated MFA system object and the input data as a DataFrame       
    """
    # --- Step 1: Define flow structure ---
    flow_definitions = all_excel_data['1_1_Definition_Flows']
    for index, row in flow_definitions.iterrows():
        if pd.notna(row['Name(EN)']):
            import re
            
            # mapping of origin and destination
            origin_id_str = str(row['Process_ID_O'])      # O = Origin = P_Start
            destination_id_str = str(row['Process_ID_I'])  # I = Input = P_End
            
            origin_match = re.match(r'(\d+)', origin_id_str) # Match the origin ID
            destination_match = re.match(r'(\d+)', destination_id_str) # Match the destination ID
            
            if origin_match and destination_match: # Ensure both matches are successful
                origin_id = int(origin_match.group(1)) # Origin becomes P_Start
                destination_id = int(destination_match.group(1)) # Destination becomes P_End

                # #Ensure both IDs are valid integers before creating the flow object
                # Ensure correct P_Start and P_End assignment

                mfa_system.FlowDict[row['Flow_ID']] = msc.Flow(
                    Name=row['Flow_ID'], 
                    P_Start=origin_id,      # Origin becomes P_Start
                    P_End=destination_id,   # Destination becomes P_End
                    Indices='t,e', 
                    Values=None
                )
    
    mfa_system.Initialize_FlowValues()
    print("--> Defined flows.")
    

    # --- Step 2: Add numerical data to flows ---
    flow_data = all_excel_data['1_2_Data_Flows']
    for flow_id, flow_obj in mfa_system.FlowDict.items():
        if flow_id in flow_data['Flow_ID'].values:
            flow_time_series = flow_data[flow_data['Flow_ID'] == flow_id]
            if len(flow_time_series) == len(mfa_system.IndexTable.Classification['Time'].Items):
                flow_obj.Values[:, 0] = np.array(flow_time_series['Flow_Py']).ravel()
    
    print("--> Added numerical data to input flows.")

    # --- Step 3: Define all Parameters ---
    parameter_id_counter = 1
    
    # Dynamic TCs - properly call the function
    dynamic_tc_sheet = all_excel_data.get('2_5_dynamic_tcs')
    if dynamic_tc_sheet is not None and not dynamic_tc_sheet.empty:
        print("--> Processing dynamic TCs...")
        time_items = mfa_system.IndexTable.Classification['Time'].Items
        
        # Call the function properly
        dynamic_tcs = create_dynamic_tc_parameters(dynamic_tc_sheet, time_items)
        
        # Add them to the system
        for name, values in dynamic_tcs.items():
            mfa_system.ParameterDict[name] = msc.Parameter(
                Name=name, 
                ID=parameter_id_counter, 
                Values=values, 
                Unit='1'
            )
            parameter_id_counter += 1
            print(f"    -> Added dynamic TC: {name}")
        
        print(f"--> Created {len(dynamic_tcs)} dynamic TC parameters")
    else:
        print("--> No dynamic TC data found or sheet is empty")
        
    # Elemental Contents
    content_definitions = all_excel_data['1_1_Definition_Flows']
    for index, row in content_definitions.iterrows():
        if pd.notna(row['Flow_ID']) and row['Flow_ID'] in mfa_system.FlowDict:
            for element in ['WC', 'DM', 'CC']:
                if element in row and pd.notna(row[element]):
                    mfa_system.ParameterDict[f"{element}_{row['Flow_ID']}"] = msc.Parameter(
                        Name=f"{element}_{row['Flow_ID']}", ID=parameter_id_counter, Values=row[element], Unit='1'
                    )
                    parameter_id_counter += 1
    
    # DSM Parameters 
    for process_id, params in dsm_params_config.items(): # Loop through each process ID in the DSM parameters configuration
        for param_name, value in params['lifetimes'].items(): # Loop through each parameter name and value
            mfa_system.ParameterDict[f"dsm_{process_id}_{param_name}"] = msc.Parameter(
                Name=f"dsm_{process_id}_{param_name}", ID=parameter_id_counter, P_Res=process_id, Values=value, Unit='yr'
            ) # Create a new parameter object
            parameter_id_counter += 1
            
    print(f"--> Defined {len(mfa_system.ParameterDict)} parameters.")

    # --- Step 4: Calculate elemental composition for primary input flows ---
    print("--> Calculating elemental composition for primary input flows...")
    for flow in mfa_system.FlowDict.values():
        # Check if the material flow for this flow is already filled
        if np.any(flow.Values[:, 0] != 0):
            # Loop through WC, DM, CC and calculate their values
            for i_elem, element_name in enumerate(mfa_system.Elements[1:], 1):
                param_name = f"{element_name}_{flow.Name}"
                if param_name in mfa_system.ParameterDict:
                    content_value = mfa_system.ParameterDict[param_name].Values
                    flow.Values[:, i_elem] = flow.Values[:, 0] * content_value
    
    # --- Final Consistency Check ---
    mfa_system.Consistency_Check()
    print("--> System setup is complete and ready for calculations!")
    
    for flow_name, flow_obj in mfa_system.FlowDict.items():
        print(f"{flow_name}: {flow_obj.Values}")

    
    return mfa_system, all_excel_data

In [12]:
# === Main Execution Block ===
#1. Modellbereich und Klassifikationen definieren
model_classification, index_table = define_model_scope(START_YEAR, END_YEAR, ELEMENTS)
# 2. Leeres System initialisieren
mfa_system = initialize_mfa_system(model_classification, index_table)
# 3. Prozesse und Lager aus Excel laden
mfa_system, input_data = load_and_define_processes(mfa_system, EXCEL_FILE_PATH)


mfa_system_configured, _ = define_flows_and_parameters(mfa_system, input_data, DSM_PARAMS)

# Check dynamic TCs
print("=== DYNAMIC TC VERIFICATION ===")
dynamic_tc_params = {name: param for name, param in mfa_system_configured.ParameterDict.items() if name.startswith('TC_')}
print(f"Total TC parameters found: {len(dynamic_tc_params)}")

for tc_name, tc_param in dynamic_tc_params.items():
    if hasattr(tc_param.Values, '__len__') and len(tc_param.Values) > 1:
        print(f"✅ Dynamic TC: {tc_name} (array length: {len(tc_param.Values)})")
    else:
        print(f"📊 Static TC: {tc_name} (value: {tc_param.Values})")


--> Model scope and classifications defined.
--> MFA system object initialized.
--> Excel file '250731_V11.xlsx' loaded successfully.

=== PROCESSING PROCESS DEFINITIONS ===
  Processing: Biosphere (ID: 0)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_0, S_0
  Processing: Raw Material Extraction (ID: 1)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Processing (ID: 2)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_2, S_2
  Processing: Use (ID: 3)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_3, S_3
  Processing: Refurbishment (ID: 4)
    TC: True, Dynamic TC: True, Stock: False
  Processing: Degredation (ID: 5)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Incineration (ID: 6)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Atmosphere (ID: 7)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_7, S_7
  Processing: Anthroposphere (ID: 8)
    TC: True, Dynamic TC:

In [13]:
# Test Function: COMPREHENSIVE TC DIAGNOSIS ===

print("=== TC PIPELINE DIAGNOSIS ===")

# Step 1: Check if dynamic_tc_sheet exists and has data
dynamic_tc_sheet = input_data.get('2_5_dynamic_tcs')
print(f"1. Dynamic TC sheet exists: {dynamic_tc_sheet is not None}")
if dynamic_tc_sheet is not None:
    print(f"   Sheet shape: {dynamic_tc_sheet.shape}")
    print(f"   Sheet empty: {dynamic_tc_sheet.empty}")
    print(f"   Columns: {list(dynamic_tc_sheet.columns)}")
    print(f"   First few rows:")
    print(dynamic_tc_sheet.head())

# Step 2: Check if the dynamic TC section is being reached
print(f"\n2. All available Excel sheets: {list(input_data.keys())}")

# Step 3: Check ALL parameters in the system
print(f"\n3. Total parameters in system: {len(mfa_system.ParameterDict)}")
print("   All parameter names:")
for name in mfa_system.ParameterDict.keys():
    param = mfa_system.ParameterDict[name]
    print(f"   - {name}: {type(param.Values)} = {param.Values}")

# Step 4: Test the function directly
print(f"\n4. Testing create_dynamic_tc_parameters directly...")
if dynamic_tc_sheet is not None and not dynamic_tc_sheet.empty:
    time_items = mfa_system.IndexTable.Classification['Time'].Items
    print(f"   Time items: {len(time_items)}")
    try:
        test_result = create_dynamic_tc_parameters(dynamic_tc_sheet, time_items)
        print(f"   Function returned: {len(test_result)} TCs")
        for name, values in test_result.items():
            print(f"   - {name}: {len(values) if hasattr(values, '__len__') else 1} values")
    except Exception as e:
        print(f"   ERROR: {e}")

=== TC PIPELINE DIAGNOSIS ===
1. Dynamic TC sheet exists: True
   Sheet shape: (37, 65)
   Sheet empty: False
   Columns: ['-', 'ID', 'Process_ID', 'Name(EN)', 'Output_Flow', 'TC?', 'DynTC?', 'Flow_ID', 'TC_ID', 'Year', 'Value', 'Stock?', 'Column1', 'Input_Flow_I_1', 'Flow_ID_I_1', 'Flow_TC_I_1_[%]', 'Input_Flow_I_2', 'Flow_ID_I_3', 'Flow_TC_I_2_[%]', 'Input_Flow_I_3', 'Flow ID I_3', 'Flow_TC_I_3_[%]', 'Input_Flow_I_4', 'Flow ID I_4', 'Flow_TC_I_4_[%]', 'Input_Flow_I_5', 'Flow ID I_5', 'Flow_TC_I_5_[%]', 'Sum_TC_I', 'Output_Flow_O_1', 'Flow_ID_O_1', 'TC_ID_O_1_Year', 'TC_ID_O_1', 'TC_O_1_[%]', 'Output_Flow_O_2', 'Flow_ID_O_2', 'TC_ID_O_2_Year', 'TC_ID_O_2', 'TC_O_2_[%]', 'Output_Flow_O_3', 'Flow_ID_O_3', 'TC_ID_O_3_Year', 'TC_ID_O_3', 'TC_O_3_[%]', 'Output_Flow_O_4', 'Flow_ID_O_4', 'TC_ID_O_4_Year', 'TC_ID_O_4', 'TC_O_4_[%]', 'Output_Flow_O_5', 'Flow_ID_O_5', 'TC_ID_O_5_Year', 'TC_ID_O_5', 'TC O_5_[%]', 'Output_Flow_O_6', 'Flow_ID_O_6', 'TC_ID_O_6_Year', 'TC_ID_O_6', 'TC O_6_[%]', 'Sum

In [14]:
# Test function: FLOW DATA VERIFICATION === 
# Check what's actually in 1_2_Data_Flows sheet

flow_data = input_data['1_2_Data_Flows']
print("=== 1_2_DATA_FLOWS CONTENT CHECK ===")
print(f"Available Flow_IDs: {list(flow_data['Flow_ID'].unique())}")
print(f"Number of data rows per flow:")
for flow_id in flow_data['Flow_ID'].unique():
    count = len(flow_data[flow_data['Flow_ID'] == flow_id])
    print(f"  {flow_id}: {count} rows")

print(f"\nExpected number of years: {len(mfa_system.IndexTable.Classification['Time'].Items)}")

# Check if F_00_01 has the right amount of data
f_00_01_data = flow_data[flow_data['Flow_ID'] == 'F_00_01']
print(f"\nF_00_01 data preview:")
print(f"  Rows: {len(f_00_01_data)}")
if not f_00_01_data.empty:
    print(f"  Sample values: {f_00_01_data['Flow_Py'].head().tolist()}")
    print(f"  Value range: {f_00_01_data['Flow_Py'].min()} to {f_00_01_data['Flow_Py'].max()}")

=== 1_2_DATA_FLOWS CONTENT CHECK ===
Available Flow_IDs: ['F_00_01']
Number of data rows per flow:
  F_00_01: 126 rows

Expected number of years: 126

F_00_01 data preview:
  Rows: 126
  Sample values: [302.6, 173.6, 186.4, 173.6, 115.8]
  Value range: 0.0 to 302.6


In [15]:
# Test function: Check Excel elemental content values

flow_defs = input_data['1_1_Definition_Flows']
print("Excel elemental content values:")
for index, row in flow_defs.iterrows():
    if pd.notna(row.get('Flow_ID')):
        flow_id = row['Flow_ID']
        cc = row.get('CC', 'N/A')
        dm = row.get('DM', 'N/A') 
        wc = row.get('WC', 'N/A')
        print(f"{flow_id}: CC={cc}, DM={dm}, WC={wc}")
        
        if cc == 1.0:
            print(f"  ❌ PROBLEM: CC=1.0 should be ~0.5!")

Excel elemental content values:
F_00_01: CC=0.5, DM=0.8, WC=0.2
F_01_02: CC=0.5, DM=0.8, WC=0.2
F_01_00: CC=0.5, DM=0.8, WC=0.2
F_02_03: CC=0.5, DM=0.8, WC=0.2
F_03_04: CC=0.5, DM=0.8, WC=0.2
F_04_03: CC=0.5, DM=0.8, WC=0.2
F_03_05: CC=0.5, DM=0.8, WC=0.2
F_03_06: CC=0.5, DM=0.8, WC=0.2
F_05_07: CC=0.5, DM=0.8, WC=0.2
F_05_00: CC=0.5, DM=0.8, WC=0.2
F_06_07: CC=0.5, DM=0.8, WC=0.2
F_06_08: CC=0.5, DM=0.8, WC=0.2
F_02_08: CC=0.5, DM=0.8, WC=0.2
F_04_08: CC=0.5, DM=0.8, WC=0.2
F_07_00: CC=0.5, DM=0.8, WC=0.2


In [16]:
# Helper Function 2.6: Calculate dynamic stocks with automatic elemental content reading.

""" 
This function calculates dynamic stocks for the MFA system using the DSM model. It automatically reads elemental contents from Excel parameters,
ensuring that the elemental composition is correctly applied to the outflow values. 
The function handles missing parameters by using default values and provides detailed results for each process. 

Args: 
    mfa_system (msc.MFASystem): The MFA system object containing processes, flows, and parameters.
    dsm_params_config (dict): Configuration dictionary for DSM parameters, including lifetimes, inflow splits, and category names.  

Returns:
    tuple: A tuple containing the modified MFA system and a dictionary with detailed results for each process.
    mfa_system (msc.MFASystem): The modified MFA system with updated flow values.
    dsm_details_results (dict): A dictionary containing detailed results for each process, including category
"""

def calculate_dynamic_stock(mfa_system, dsm_params_config):
    """
    Enhanced DSM calculation with automatic elemental content reading from Excel
    """
    print("--> Calculating dynamic stocks with automatic elemental content...")
    time_vector = np.array(mfa_system.IndexTable.Classification['Time'].Items)
    dsm_details_results = {} 

    # ✅ AUTO-READ ELEMENTAL CONTENTS FROM EXCEL
    def get_elemental_content_from_excel(flow_name, element):
        """Helper function to read elemental content from Excel parameters"""
        param_name = f"{element}_{flow_name}"
        if param_name in mfa_system.ParameterDict:
            return float(mfa_system.ParameterDict[param_name].Values)
        else:
            # Default values if not found in Excel
            defaults = {'CC': 0.5, 'DM': 0.9, 'WC': 0.1}
            print(f"⚠️ Warning: {param_name} not found, using default {element}={defaults[element]}")
            return defaults[element]

    for process_id, params in dsm_params_config.items():
        if not any(p.ID == process_id for p in mfa_system.ProcessList):
            print(f"Warning: Process {process_id} from DSM_PARAMS not found in model. Skipping.")
            continue

        print(f"    ... running DSM for process {process_id}")
        dsm_details_results[process_id] = {'category_names': [], 'stock_by_category': []}

        # Get inflow and outflow flow objects
        inflow_flow = next((f for f in mfa_system.FlowDict.values() if f.P_End == process_id), None)
        outflow_flow = next((f for f in mfa_system.FlowDict.values() if f.P_Start == process_id), None)
        
        if not inflow_flow or not outflow_flow:
            print(f"Warning: Missing inflow or outflow for process {process_id}")
            continue

        inflow_values = inflow_flow.Values # Materil flow over time
        if inflow_values.ndim == 1:
            inflow_values = inflow_values.reshape(-1, 1)

        # ✅ AUTO-READ ELEMENTAL CONTENTS FOR THIS FLOW
        inflow_cc_content = get_elemental_content_from_excel(inflow_flow.Name, 'CC')
        inflow_dm_content = get_elemental_content_from_excel(inflow_flow.Name, 'DM') 
        inflow_wc_content = get_elemental_content_from_excel(inflow_flow.Name, 'WC')
        
        print(f"      Auto-read elemental contents for {inflow_flow.Name}:")
        print(f"        CC: {inflow_cc_content:.1%}")
        print(f"        DM: {inflow_dm_content:.1%}")
        print(f"        WC: {inflow_wc_content:.1%}")

        # DSM parameters
        lt_params = params.get('lifetimes', {}) # Ensure lt_params is a dict
        inflow_split = params.get('inflow_split', [1.0]) # Default to [1.0] if not provided
        category_names = params.get('category_names', []) # Default to empty list if not provided
        mean_lifetimes = lt_params.get('Mean', []) # Default to empty list if not provided
        std_devs = lt_params.get('StdDev', []) # Default to empty list if not provided

        category_labels = [f"{name} ({lt} yrs)" for name, lt in zip(category_names, mean_lifetimes)] # Create category labels with lifetimes
        dsm_details_results[process_id]['category_names'] = category_labels

        # Initialize arrays
        outflow_total_material = np.zeros(len(time_vector))
        stock_total_material = np.zeros(len(time_vector))
        stock_by_category_list = []

        # DSM calculation for each category
        for i in range(len(inflow_split)): 
            inflow_category = inflow_values[:, 0] * inflow_split[i]

            if np.sum(inflow_category) > 0: #
                dsm_model = dsm.DynamicStockModel(
                    t=time_vector, # Years 1920-2045
                    i=inflow_category, # Annual inflow
                    lt={'Type': lt_params.get('Type'),  # Lifetime type
                        'Mean': [mean_lifetimes[i]],  # Mean lifetime for this category
                        'StdDev': [std_devs[i]]} #  Standard deviation for this category
                )
                s_c = dsm_model.compute_s_c_inflow_driven() # Calculate stock by cohort (s_c)
                o_c = dsm_model.compute_o_c_from_s_c() # Calculate outflow by cohort (o_c) 

                if o_c is not None: # Aggregate across all cohorts
                    outflow_total_material += o_c.sum(axis=1)
                    stock_category_ts = s_c.sum(axis=1)
                    stock_total_material += stock_category_ts
                    stock_by_category_list.append(stock_category_ts)
                else:
                    stock_by_category_list.append(np.zeros(len(time_vector)))
            else:
                stock_by_category_list.append(np.zeros(len(time_vector)))

        dsm_details_results[process_id]['stock_by_category'] = stock_by_category_list

        # ✅ FIXED: Assign outflow with FIXED elemental composition
        outflow_flow.Values[:, 0] = outflow_total_material
        
        # Calculate elemental flows using FIXED composition from Excel
        element_contents = {
            'CC': inflow_cc_content,
            'DM': inflow_dm_content, 
            'WC': inflow_wc_content
        }
        
        for idx_elem, element in enumerate(mfa_system.Elements[1:], 1):
            if element in element_contents:
                # Use fixed elemental content
                content_value = element_contents[element]
                outflow_flow.Values[:, idx_elem] = outflow_total_material * content_value
                print(f"        {element} outflow calculated with fixed content {content_value:.1%}")
            else:
                print(f"        ⚠️ Unknown element {element}, setting to zero")
                outflow_flow.Values[:, idx_elem] = 0

        # Calculate stock changes and absolute stocks with fixed composition
        dS_values = inflow_values - outflow_flow.Values
        mfa_system.StockDict[f'dS_{process_id}'].Values = dS_values

        # ✅ FIXED: Calculate stocks with fixed elemental composition
        mfa_system.StockDict[f'S_{process_id}'].Values[:, 0] = stock_total_material
        
        for idx_elem, element in enumerate(mfa_system.Elements[1:], 1):
            if element in element_contents:
                content_value = element_contents[element]
                stock_total_element = stock_total_material * content_value
                # Prevent negative stocks
                stock_total_element = np.maximum(stock_total_element, 0)
                mfa_system.StockDict[f'S_{process_id}'].Values[:, idx_elem] = stock_total_element
            else:
                mfa_system.StockDict[f'S_{process_id}'].Values[:, idx_elem] = 0

        print(f"      ✅ DSM completed for Process {process_id}")

    print("--> Dynamic stock calculation finished.")
    return mfa_system, dsm_details_results

In [17]:
#  Test Function for calculate_dynamic_stock 

def test_calculate_dynamic_stock():
    """
    Simple test function for calculate_dynamic_stock.
    """
    print("=== Testing calculate_dynamic_stock ===")
    
    try:
        # Setup test environment
        model_classification, index_table = define_model_scope(START_YEAR, END_YEAR, ELEMENTS)
        test_mfa_system = initialize_mfa_system(model_classification, index_table)
        test_mfa_system, excel_data = load_and_define_processes(test_mfa_system, EXCEL_FILE_PATH)
        test_mfa_system, _ = define_flows_and_parameters(test_mfa_system, excel_data, DSM_PARAMS)
        
        # Test the function
        updated_system, dsm_details = calculate_dynamic_stock(test_mfa_system, DSM_PARAMS)
        
        # # Show results
        # print(f"✅ Function executed successfully!")
        # print(f"   - DSM processes calculated: {len(dsm_details)}")
        
        # Show stock information
        print("\n=== STOCK RESULTS ===")
        for stock_name, stock_obj in updated_system.StockDict.items():
            if stock_obj.Values is not None:
                print(f"Stock {stock_name}:")
                print(f"   - Shape: {stock_obj.Values.shape}")
                print(f"   - Min: {stock_obj.Values.min():.2f}")
                print(f"   - Max: {stock_obj.Values.max():.2f}")
                print(f"   - Final year total: {stock_obj.Values[-1, :].sum():.2f}")
        
        # Show DSM details
        print("\n=== DSM DETAILS ===")
        for process_id, details in dsm_details.items():
            print(f"Process {process_id}:")
            print(f"   - Categories: {details['category_names']}")
            print(f"   - Stock categories: {len(details['stock_by_category'])}")
        
        return True
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# === EXECUTE THE TEST ===
test_calculate_dynamic_stock()

=== Testing calculate_dynamic_stock ===
--> Model scope and classifications defined.
--> MFA system object initialized.
--> Excel file '250731_V11.xlsx' loaded successfully.

=== PROCESSING PROCESS DEFINITIONS ===
  Processing: Biosphere (ID: 0)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_0, S_0
  Processing: Raw Material Extraction (ID: 1)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Processing (ID: 2)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_2, S_2
  Processing: Use (ID: 3)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_3, S_3
  Processing: Refurbishment (ID: 4)
    TC: True, Dynamic TC: True, Stock: False
  Processing: Degredation (ID: 5)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Incineration (ID: 6)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Atmosphere (ID: 7)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_7, S_7
  Processing: Anthropo

True

In [18]:
# Helper Function 2.7: Calculate final stock changes and balances

def calculate_final_balances(mfa_system):
    """
    Calculates the stock changes (dS) and absolute stocks (S) for all processes.
    Skip DSM processes to avoid overwriting their calculated stocks.

    Args:
        mfa_system (msc.MFASystem): The MFA system object containing processes, flows, and stocks.  

    Returns:
        mfa_system (msc.MFASystem): The modified MFA system with updated stock values
    """
    print("--> Calculating final stock balances...")
    
    # ✅ CRITICAL FIX: Skip both primary inputs AND DSM processes
    PRIMARY_INPUT_PROCESSES = [0]  # Process 0 is Biosphere
    DSM_PROCESSES = list(DSM_PARAMS.keys()) if 'DSM_PARAMS' in globals() else []
    SKIP_PROCESSES = PRIMARY_INPUT_PROCESSES + DSM_PROCESSES
    
    for process in mfa_system.ProcessList:
        # ✅ SKIP both primary inputs and DSM processes
        if process.ID in SKIP_PROCESSES:
            process_type = "primary input" if process.ID in PRIMARY_INPUT_PROCESSES else "DSM"
            print(f"  -> Skipping balance calculation for {process_type} process {process.ID} ({process.Name})")
            continue
            
        # Check if a stock is associated with this process
        if f"S_{process.ID}" in mfa_system.StockDict:
            
            # Find all input and output flows for this process
            input_flows = [f.Values for f in mfa_system.FlowDict.values() if f.P_End == process.ID]
            output_flows = [f.Values for f in mfa_system.FlowDict.values() if f.P_Start == process.ID]
            
            sum_input_flows = sum(input_flows) if input_flows else 0
            sum_output_flows = sum(output_flows) if output_flows else 0
            
            # Calculate stock change (dS) and absolute stock (S)
            dS_values = sum_input_flows - sum_output_flows
            mfa_system.StockDict[f"dS_{process.ID}"].Values = dS_values
            mfa_system.StockDict[f"S_{process.ID}"].Values = dS_values.cumsum(axis=0)
            
            print(f"  -> Calculated balance for Process {process.ID} ({process.Name})")
            
    print("--> Stock balance calculation finished.")
    return mfa_system

In [19]:
# --- Helper Function 2.8: Plot Mass Balance Error per Process  ---

def plot_mass_balance_error(mfa_system_results):
    """
    Creates an interactive bar chart showing the mass balance error for each process.
    Error = Inflows - Outflows - dS. An error of 0 means perfect balance.

    Args:
        mfa_system_results (msc.MFASystem): The MFA system object after calculations, containing processes, flows, and stocks.
    
    Returns:
        None: Displays an interactive plot.
    """

    process_names = [p.Name for p in mfa_system_results.ProcessList]
    time_items = mfa_system_results.IndexTable.Classification['Time'].Items
    element_items = mfa_system_results.Elements
    
    fig = go.FigureWidget()

    def update_plot(year, element):
        year_index = time_items.index(year)
        element_index = element_items.index(element)
        
        errors = []
        for p in mfa_system_results.ProcessList:
            in_val = sum(f.Values[year_index, element_index] for f in mfa_system_results.FlowDict.values() if f.P_End == p.ID)
            out_val = sum(f.Values[year_index, element_index] for f in mfa_system_results.FlowDict.values() if f.P_Start == p.ID)
            ds_val = mfa_system_results.StockDict.get(f'dS_{p.ID}', None)
            ds_sum = ds_val.Values[year_index, element_index] if ds_val is not None else 0
            
            error = in_val - out_val - ds_sum
            errors.append(error)
        
        # Color bars based on error direction
        colors = ['#d62728' if e > 1e-9 else '#2ca02c' if e < -1e-9 else '#7f7f7f' for e in errors] # Red for positive, Green for negative, Grey for zero

        with fig.batch_update():
            fig.data = [] # Clear previous data
            fig.add_trace(go.Bar(x=process_names, y=errors, marker_color=colors))
            fig.update_layout(
                title=f"Mass Balance Error Check for {element.upper()} in {year}",
                yaxis_title="Error in Mg (positive = mass created)",
                shapes=[dict(type='line', y0=0, y1=0, x0=-0.5, x1=len(process_names)-0.5, line=dict(color='black', width=2))] # Zero line
            )

    # Create widgets
    year_slider = IntSlider(min=time_items[0], max=time_items[-1], step=1, value=time_items[0], description='Year')
    element_dropdown = Dropdown(options=element_items, value=element_items[0], description='Element:')
    
    interact(update_plot, year=year_slider, element=element_dropdown)
    display(fig)

In [20]:
# Main Solver Function with Dynamic TC Handling ---

def run_mfa_calculation(mfa_system_configured):
    """
    Enhanced MFA solver that properly handles dynamic TCs for DSM outputs

    Args:
        mfa_system_configured (msc.MFASystem): The configured MFA system object with processes, flows, and parameters.
    
    Returns:
        tuple: A tuple containing the modified MFA system and a dictionary with detailed DSM results.
        mfa_system (msc.MFASystem): The modified MFA system with updated flow values.
        dsm_details (dict): A dictionary containing detailed results for each DSM process.  
    """
    import copy
    import numpy as np

    mfa_system = copy.deepcopy(mfa_system_configured)
    dsm_details = {}

    # Get primary flows and DSM processes
    PRIMARY_INPUT_FLOWS = [f.Name for f in mfa_system.FlowDict.values() if f.P_Start == 0]
    dsm_processes = set(DSM_PARAMS.keys()) if 'DSM_PARAMS' in globals() else set()
    dsm_processes_run = {pid: False for pid in dsm_processes}

    # Backup primary flows
    original_primary_data = {}
    for flow_name in PRIMARY_INPUT_FLOWS:
        if flow_name in mfa_system.FlowDict and mfa_system.FlowDict[flow_name].Values is not None:
            original_primary_data[flow_name] = mfa_system.FlowDict[flow_name].Values.copy()

    print("🚀 STARTING ENHANCED MFA SOLVER WITH DYNAMIC TC SUPPORT")
    print(f"Primary flows: {PRIMARY_INPUT_FLOWS}")
    print(f"DSM processes: {dsm_processes}")

    for iteration in range(50):
        print(f"\n--- Iteration {iteration+1} ---")
        something_changed = False
        flows_calculated_this_iteration = []

        # Always restore primary flows
        for flow_name in PRIMARY_INPUT_FLOWS:
            if flow_name in mfa_system.FlowDict and flow_name in original_primary_data:
                mfa_system.FlowDict[flow_name].Values = original_primary_data[flow_name].copy()

        # 1. TC-based flow calculation
        all_flows = list(mfa_system.FlowDict.values())
        all_flows.sort(key=lambda f: f.P_Start)
        
        for flow in all_flows:
            if flow.Name in PRIMARY_INPUT_FLOWS:
                continue  # Skip primary inputs
                
            # Only calculate if flow is currently zero
            if flow.Values is not None and np.all(flow.Values == 0):
                flow_parts = flow.Name.split('_')
                if len(flow_parts) >= 3:
                    param_name = f"TC_{flow_parts[1]}_{flow_parts[2]}"
                else:
                    continue
                
                if param_name in mfa_system.ParameterDict:
                    # ✅ KEY FIX: Handle flows FROM DSM processes
                    if flow.P_Start in dsm_processes:
                        # For DSM outputs, we need the DSM total output as input
                        dsm_output_flows = [f for f in mfa_system.FlowDict.values() 
                                          if f.P_Start == flow.P_Start and np.any(f.Values != 0)]
                        
                        if dsm_output_flows:
                            # Use the DSM-calculated output as input for TC calculation
                            dsm_total_output = sum(f.Values[:, 0] for f in dsm_output_flows)
                            
                            print(f"  ✅ Calculating DSM output {flow.Name} using {param_name}")
                            
                            # Get TC value (handle dynamic TCs)
                            tc_param = mfa_system.ParameterDict[param_name]
                            if hasattr(tc_param.Values, '__len__') and len(tc_param.Values) > 1:
                                tc_value = tc_param.Values  # Dynamic TC
                            else:
                                tc_value = tc_param.Values  # Static TC

                            # Calculate flow
                            flow.Values[:, 0] = dsm_total_output * tc_value
                            something_changed = True
                            flows_calculated_this_iteration.append(flow.Name)

                            # Calculate elemental flows
                            for idx_elem, element in enumerate(mfa_system.Elements[1:], 1):
                                content_param = f"{element}_{flow.Name}"
                                if content_param in mfa_system.ParameterDict:
                                    content_value = mfa_system.ParameterDict[content_param].Values
                                    flow.Values[:, idx_elem] = flow.Values[:, 0] * content_value

                            print(f"      Input flows ready: DSM output from Process {flow.P_Start}")
                            print(f"      max={flow.Values.max():.2f}, sum={flow.Values.sum():.2f}")
                    else:
                        # Regular TC calculation for non-DSM flows
                        input_flows = [f for f in mfa_system.FlowDict.values() if f.P_End == flow.P_Start]
                        ready_input_flows = [f for f in input_flows if f.Values is not None and np.any(f.Values != 0)]
                        
                        if ready_input_flows:
                            print(f"  ✅ Calculating {flow.Name} using {param_name}")
                            print(f"      Input flows ready: {[f.Name for f in ready_input_flows]}")
                            
                            sum_input_flows = np.sum([f.Values[:, 0] for f in ready_input_flows], axis=0)
                            
                            tc_param = mfa_system.ParameterDict[param_name]
                            if hasattr(tc_param.Values, '__len__') and len(tc_param.Values) > 1:
                                tc_value = tc_param.Values
                            else:
                                tc_value = tc_param.Values

                            flow.Values[:, 0] = sum_input_flows * tc_value
                            something_changed = True
                            flows_calculated_this_iteration.append(flow.Name)

                            for idx_elem, element in enumerate(mfa_system.Elements[1:], 1):
                                content_param = f"{element}_{flow.Name}"
                                if content_param in mfa_system.ParameterDict:
                                    content_value = mfa_system.ParameterDict[content_param].Values
                                    flow.Values[:, idx_elem] = flow.Values[:, 0] * content_value

                            print(f"      max={flow.Values.max():.2f}, sum={flow.Values.sum():.2f}")
                        else:
                            missing_inputs = [f.Name for f in input_flows if f.Values is None or np.all(f.Values == 0)]
                            if missing_inputs:
                                print(f"  ⏳ {flow.Name} waiting for inputs: {missing_inputs}")

        print(f"  Calculated flows this iteration: {flows_calculated_this_iteration}")

        # 2. DSM calculation
        if globals().get('RUN_DSM_CALCULATION', False):
            for pid in dsm_processes:
                if not dsm_processes_run[pid]:
                    inflows = [f for f in mfa_system.FlowDict.values() if f.P_End == pid]
                    ready_inflows = [f for f in inflows if f.Values is not None and np.any(f.Values != 0)]
                    
                    if ready_inflows:
                        print(f"--> 🤖 Running DSM for Process {pid}...")
                        try:
                            mfa_system, dsm_details_single = calculate_dynamic_stock(mfa_system, {pid: DSM_PARAMS[pid]})
                            dsm_details.update(dsm_details_single)
                            dsm_processes_run[pid] = True
                            something_changed = True
                            
                            # Verify DSM outputs
                            outflows = [f for f in mfa_system.FlowDict.values() if f.P_Start == pid]
                            for outflow in outflows:
                                if outflow.Values is not None and np.any(outflow.Values != 0):
                                    print(f"      ✅ DSM Output {outflow.Name}: max={outflow.Values.max():.2f}")
                                    
                        except Exception as e:
                            print(f"❌ Error running DSM for {pid}: {e}")

        # Always restore primary flows at the end
        for flow_name in PRIMARY_INPUT_FLOWS:
            if flow_name in mfa_system.FlowDict and flow_name in original_primary_data:
                mfa_system.FlowDict[flow_name].Values = original_primary_data[flow_name].copy()

        # Check convergence
        if not something_changed:
            print(f"--- ✅ System converged at iteration {iteration+1} ---")
            break
        elif iteration == 49:
            print("--- ⚠️ Reached maximum iterations ---")

    # Final summary
    print("\n--- FINAL FLOW STATUS ---")
    flows_with_data = []
    flows_without_data = []
    
    for flow_name, flow_obj in mfa_system.FlowDict.items():
        if np.any(flow_obj.Values != 0):
            flows_with_data.append(flow_name)
        else:
            flows_without_data.append(flow_name)
    
    print(f"✅ Flows WITH data ({len(flows_with_data)}): {flows_with_data}")
    print(f"❌ Flows WITHOUT data ({len(flows_without_data)}): {flows_without_data}")
    
    # Final balance calculation
    print("\n--- Finalizing stock balances ---")
    mfa_system = calculate_final_balances(mfa_system)
    
    print(f"\n🎉 MFA calculation finished after {iteration+1} iterations.")
    return mfa_system, dsm_details



In [21]:
# Run the MFA calculation
print("=== RUNNING ENHANCED MFA SOLVER WITH DYNAMIC TC SUPPORT ===")
mfa_system_with_results, dsm_details = run_mfa_calculation(mfa_system_configured)

# Check which flows now have calculated values
print("\n=== AFTER MFA CALCULATION ===")
non_zero_flows = 0
for flow_name, flow_obj in mfa_system_with_results.FlowDict.items():
    max_val = flow_obj.Values.max()
    if max_val > 0:
        non_zero_flows += 1
        print(f"✅ {flow_name}: max={max_val:.2f}, sum={flow_obj.Values.sum():.2f}")

print(f"\n📊 Summary: {non_zero_flows} flows have calculated values")

=== RUNNING ENHANCED MFA SOLVER WITH DYNAMIC TC SUPPORT ===
🚀 STARTING ENHANCED MFA SOLVER WITH DYNAMIC TC SUPPORT
Primary flows: ['F_00_01']
DSM processes: {0, 2, 3, 7, 8}

--- Iteration 1 ---
  ✅ Calculating F_01_02 using TC_01_02
      Input flows ready: ['F_00_01']
      max=211.82, sum=11949.35
  ✅ Calculating F_01_00 using TC_01_00
      Input flows ready: ['F_00_01']
      max=90.78, sum=5121.15
  ⏳ F_04_03 waiting for inputs: ['F_03_04']
  ⏳ F_04_08 waiting for inputs: ['F_03_04']
  ⏳ F_05_07 waiting for inputs: ['F_03_05']
  ⏳ F_05_00 waiting for inputs: ['F_03_05']
  ⏳ F_06_07 waiting for inputs: ['F_03_06']
  ⏳ F_06_08 waiting for inputs: ['F_03_06']
  Calculated flows this iteration: ['F_01_02', 'F_01_00']
--> 🤖 Running DSM for Process 0...
--> Calculating dynamic stocks with automatic elemental content...
    ... running DSM for process 0
      Auto-read elemental contents for F_01_00:
        CC: 50.0%
        DM: 80.0%
        WC: 20.0%
        WC outflow calculated with

In [22]:
# Debug flow calculation process

print("=== FLOW CALCULATION DEBUG ===")

# Check which flows have data vs. which are empty
flows_with_data = []
flows_without_data = []

for flow_name, flow_obj in mfa_system_with_results.FlowDict.items():
    if np.any(flow_obj.Values != 0):
        flows_with_data.append(flow_name)
    else:
        flows_without_data.append(flow_name)

print(f"✅ Flows WITH data ({len(flows_with_data)}):")
for flow in flows_with_data:
    print(f"  {flow}")

print(f"\n❌ Flows WITHOUT data ({len(flows_without_data)}):")
for flow in flows_without_data:
    print(f"  {flow}")

# Check which TC parameters exist
print(f"\n=== TRANSFER COEFFICIENT CHECK ===")
tc_params = {name: param for name, param in mfa_system_with_results.ParameterDict.items() if name.startswith('TC_')}
print(f"Available TC parameters ({len(tc_params)}):")
for tc_name in sorted(tc_params.keys()):
    print(f"  {tc_name}")

# For each missing flow, check if its TC exists
print(f"\n=== MISSING TC ANALYSIS ===")
for flow_name in flows_without_data:
    expected_tc = f"TC_{'_'.join(flow_name.split('_')[1:3])}"
    if expected_tc in tc_params:
        print(f"✅ {flow_name} -> {expected_tc} EXISTS")
    else:
        print(f"❌ {flow_name} -> {expected_tc} MISSING")

# Check DSM configuration
print(f"\n=== DSM PROCESS CHECK ===")
print(f"DSM_PARAMS processes: {list(DSM_PARAMS.keys())}")
print(f"RUN_DSM_CALCULATION: {globals().get('RUN_DSM_CALCULATION', 'NOT SET')}")

for process_id in DSM_PARAMS.keys():
    inflows = [f.Name for f in mfa_system_with_results.FlowDict.values() if f.P_End == process_id]
    outflows = [f.Name for f in mfa_system_with_results.FlowDict.values() if f.P_Start == process_id]
    print(f"Process {process_id}:")
    print(f"  Inflows: {inflows}")
    print(f"  Outflows: {outflows}")
    
    # Check if inflows have data
    for inflow_name in inflows:
        inflow_obj = mfa_system_with_results.FlowDict[inflow_name]
        has_data = np.any(inflow_obj.Values != 0)
        print(f"    {inflow_name} has data: {has_data}")

=== FLOW CALCULATION DEBUG ===
✅ Flows WITH data (15):
  F_00_01
  F_01_02
  F_01_00
  F_02_03
  F_03_04
  F_04_03
  F_03_05
  F_03_06
  F_05_07
  F_05_00
  F_06_07
  F_06_08
  F_02_08
  F_04_08
  F_07_00

❌ Flows WITHOUT data (0):

=== TRANSFER COEFFICIENT CHECK ===
Available TC parameters (15):
  TC_00_01
  TC_01_00
  TC_01_02
  TC_02_03
  TC_02_08
  TC_03_04
  TC_03_05
  TC_03_06
  TC_04_03
  TC_04_08
  TC_05_00
  TC_05_07
  TC_06_07
  TC_06_08
  TC_07_00

=== MISSING TC ANALYSIS ===

=== DSM PROCESS CHECK ===
DSM_PARAMS processes: [0, 2, 3, 7, 8]
RUN_DSM_CALCULATION: True
Process 0:
  Inflows: ['F_01_00', 'F_05_00', 'F_07_00']
  Outflows: ['F_00_01']
    F_01_00 has data: True
    F_05_00 has data: True
    F_07_00 has data: True
Process 2:
  Inflows: ['F_01_02']
  Outflows: ['F_02_03', 'F_02_08']
    F_01_02 has data: True
Process 3:
  Inflows: ['F_02_03', 'F_04_03']
  Outflows: ['F_03_04', 'F_03_05', 'F_03_06']
    F_02_03 has data: True
    F_04_03 has data: True
Process 7:
  In

In [23]:
# Re-run the MFA calculation with the enhanced solver
mfa_system_with_results, dsm_details = run_mfa_calculation(mfa_system_configured)

# Check results
print("\n=== UPDATED FLOW SUMMARY ===")
for flow_name, flow_obj in mfa_system_with_results.FlowDict.items():
    max_val = flow_obj.Values.max()
    sum_val = flow_obj.Values.sum()
    has_data = "✅" if max_val > 0 else "❌"
    print(f"{has_data} {flow_name}: max={max_val:.2f}, sum={sum_val:.2f}")

🚀 STARTING ENHANCED MFA SOLVER WITH DYNAMIC TC SUPPORT
Primary flows: ['F_00_01']
DSM processes: {0, 2, 3, 7, 8}

--- Iteration 1 ---
  ✅ Calculating F_01_02 using TC_01_02
      Input flows ready: ['F_00_01']
      max=211.82, sum=11949.35
  ✅ Calculating F_01_00 using TC_01_00
      Input flows ready: ['F_00_01']
      max=90.78, sum=5121.15
  ⏳ F_04_03 waiting for inputs: ['F_03_04']
  ⏳ F_04_08 waiting for inputs: ['F_03_04']
  ⏳ F_05_07 waiting for inputs: ['F_03_05']
  ⏳ F_05_00 waiting for inputs: ['F_03_05']
  ⏳ F_06_07 waiting for inputs: ['F_03_06']
  ⏳ F_06_08 waiting for inputs: ['F_03_06']
  Calculated flows this iteration: ['F_01_02', 'F_01_00']
--> 🤖 Running DSM for Process 0...
--> Calculating dynamic stocks with automatic elemental content...
    ... running DSM for process 0
      Auto-read elemental contents for F_01_00:
        CC: 50.0%
        DM: 80.0%
        WC: 20.0%
        WC outflow calculated with fixed content 20.0%
        DM outflow calculated with fixe

In [24]:
# Check mass balance errors where process 0 is being excluded
print("=== MASS BALANCE ANALYSIS ===")
balance = mfa_system_with_results.MassBalance()
print("Mass balance errors (should be close to zero):")
print(np.abs(balance).sum(axis=0).sum(axis=1))

# Check individual process balances - EXCLUDING PROCESS 0
for process in mfa_system_with_results.ProcessList:
    pid = process.ID
    
    # SKIP Process 0 (Biosphere) - it's a system boundary, not a mass-conserving process
    if pid == 0:
        print(f"Process {pid} ({process.Name}): SKIPPED (system boundary)")
        continue
    
    inflows = sum(f.Values for f in mfa_system_with_results.FlowDict.values() if f.P_End == pid)
    outflows = sum(f.Values for f in mfa_system_with_results.FlowDict.values() if f.P_Start == pid)
    
    if f"S_{pid}" in mfa_system_with_results.StockDict:
        stock_change = mfa_system_with_results.StockDict[f"dS_{pid}"].Values
        balance_error = inflows - outflows - stock_change
        max_error = np.abs(balance_error).max()
        print(f"Process {pid} ({process.Name}): Max error = {max_error:.6f} Mg")

=== MASS BALANCE ANALYSIS ===
Mass balance errors (should be close to zero):
[5.20289941e+03 1.24111832e-12 4.53763524e+03 4.71460312e+03
 2.69248516e-13 1.29515405e-14 2.68546983e-13 4.55265177e+03
 9.90248600e+03]
Process 0 (Biosphere): SKIPPED (system boundary)
Process 2 (Processing): Max error = 54.153680 Mg
Process 3 (Use): Max error = 38.083195 Mg
Process 7 (Atmosphere): Max error = 36.913888 Mg
Process 8 (Anthroposphere): Max error = 54.153785 Mg


In [25]:
# Analyze DSM lifetimes for realism
print("\n=== DSM LIFETIME ANALYSIS ===")
for process_id, params in DSM_PARAMS.items():
    lifetimes = params['lifetimes']['Mean']
    categories = params['category_names']
    
    print(f"\nProcess {process_id}:")
    for i, (category, lifetime) in enumerate(zip(categories, lifetimes)):
        print(f"  {category}: {lifetime} years")
        
        # Reality check for timber products
        if process_id == 3:  # Use phase
            if lifetime < 10:
                print(f"    ⚠️ WARNING: {lifetime} years seems short for timber products")
            elif lifetime > 100:
                print(f"    ⚠️ WARNING: {lifetime} years seems long for typical construction")
            else:
                print(f"    ✅ Realistic lifespan for timber construction")


=== DSM LIFETIME ANALYSIS ===

Process 0:
  Average Exchange: 20 years

Process 2:
  Air Drying: 6 years

Process 3:
  Average Lifetime: 80 years
    ✅ Realistic lifespan for timber construction

Process 7:
  Photosynthesis: 70 years

Process 8:


In [26]:
# Analyze stock accumulation patterns
print("\n=== STOCK ACCUMULATION ANALYSIS ===")
for stock_name, stock_obj in mfa_system_with_results.StockDict.items():
    if stock_name.startswith('S_') and np.any(stock_obj.Values != 0):
        final_stock = stock_obj.Values[-1, 0]  # Final year, material element
        max_stock = stock_obj.Values[:, 0].max()
        
        print(f"{stock_name}:")
        print(f"  Final stock (2045): {final_stock:.2f} Mg")
        print(f"  Peak stock: {max_stock:.2f} Mg")
        
        # Check for unrealistic exponential growth
        if final_stock > max_stock * 0.9:  # Still growing at end
            print(f"    ⚠️ WARNING: Stock still growing rapidly at model end")
        
        # Check growth rate in last decade
        if len(stock_obj.Values) >= 10:
            growth_rate = (stock_obj.Values[-1, 0] / stock_obj.Values[-10, 0]) ** (1/10) - 1
            print(f"  Annual growth rate (last 10 years): {growth_rate*100:.1f}%")
            if growth_rate > 0.1:  # >10% annual growth
                print(f"    ⚠️ WARNING: Very high growth rate")


=== STOCK ACCUMULATION ANALYSIS ===
S_0:
  Final stock (2045): 69.43 Mg
  Peak stock: 555.47 Mg
  Annual growth rate (last 10 years): -2.7%
S_2:
  Final stock (2045): 57.96 Mg
  Peak stock: 673.10 Mg
  Annual growth rate (last 10 years): 2.7%
S_3:
  Final stock (2045): 2554.45 Mg
  Peak stock: 3618.65 Mg
  Annual growth rate (last 10 years): -1.2%
S_7:
  Final stock (2045): 79.84 Mg
  Peak stock: 79.84 Mg
    ⚠️ WARNING: Stock still growing rapidly at model end
  Annual growth rate (last 10 years): 0.9%


In [27]:
# Add this diagnostic cell to understand what's happening

print("=== DETAILED STOCK DIAGNOSTIC ===")

# Check the stock calculation logic
for process in mfa_system_with_results.ProcessList:
    pid = process.ID
    
    if f"S_{pid}" in mfa_system_with_results.StockDict:
        print(f"\n--- Process {pid} ({process.Name}) ---")
        
        # Get inflows and outflows
        inflows = [f for f in mfa_system_with_results.FlowDict.values() if f.P_End == pid]
        outflows = [f for f in mfa_system_with_results.FlowDict.values() if f.P_Start == pid]
        
        print(f"Inflows: {[f.Name for f in inflows]}")
        print(f"Outflows: {[f.Name for f in outflows]}")
        
        # Calculate totals for final year (2045)
        total_inflow = sum(f.Values[-1, 0] for f in inflows)
        total_outflow = sum(f.Values[-1, 0] for f in outflows)
        final_stock = mfa_system_with_results.StockDict[f"S_{pid}"].Values[-1, 0]
        
        print(f"Final year (2045):")
        print(f"  Total inflow: {total_inflow:.2f} Mg")
        print(f"  Total outflow: {total_outflow:.2f} Mg")
        print(f"  Final stock: {final_stock:.2f} Mg")
        print(f"  Balance (In-Out): {total_inflow - total_outflow:.2f} Mg")
        
        # Check for missing outflows
        if total_inflow > 0 and total_outflow == 0:
            print(f"  ⚠️ WARNING: Inflows but no outflows - missing flow definition?")
        
        if final_stock < 0:
            print(f"  ❌ CRITICAL: Negative stock detected!")

=== DETAILED STOCK DIAGNOSTIC ===

--- Process 0 (Biosphere) ---
Inflows: ['F_01_00', 'F_05_00', 'F_07_00']
Outflows: ['F_00_01']
Final year (2045):
  Total inflow: 2.22 Mg
  Total outflow: 6.40 Mg
  Final stock: 69.43 Mg
  Balance (In-Out): -4.18 Mg

--- Process 2 (Processing) ---
Inflows: ['F_01_02']
Outflows: ['F_02_03', 'F_02_08']
Final year (2045):
  Total inflow: 4.48 Mg
  Total outflow: 9.79 Mg
  Final stock: 57.96 Mg
  Balance (In-Out): -5.31 Mg

--- Process 3 (Use) ---
Inflows: ['F_02_03', 'F_04_03']
Outflows: ['F_03_04', 'F_03_05', 'F_03_06']
Final year (2045):
  Total inflow: 8.26 Mg
  Total outflow: 76.75 Mg
  Final stock: 2554.45 Mg
  Balance (In-Out): -68.49 Mg

--- Process 7 (Atmosphere) ---
Inflows: ['F_05_07', 'F_06_07']
Outflows: ['F_07_00']
Final year (2045):
  Total inflow: 34.10 Mg
  Total outflow: 0.26 Mg
  Final stock: 79.84 Mg
  Balance (In-Out): 33.84 Mg

--- Process 8 (Anthroposphere) ---
Inflows: ['F_06_08', 'F_02_08', 'F_04_08']
Outflows: []
Final year (2045

In [28]:
print("=== DSM OUTFLOW VERIFICATION ===")

for process_id in [2, 3]:  # Your DSM processes with negative stocks
    print(f"\nProcess {process_id}:")
    
    # Get inflow data
    inflows = [f for f in mfa_system_with_results.FlowDict.values() if f.P_End == process_id]
    if not inflows:
        print(f"  ❌ No inflows found for process {process_id}")
        continue
        
    # FIXED: Calculate total cumulative inflow correctly
    total_cumulative_inflow = sum(f.Values[:, 0].sum() for f in inflows)
    
    # Get outflow data  
    outflows = [f for f in mfa_system_with_results.FlowDict.values() if f.P_Start == process_id]
    if not outflows:
        print(f"  ❌ No outflows found for process {process_id}")
        continue
        
    # FIXED: Calculate total cumulative outflow correctly
    total_cumulative_outflow = sum(f.Values[:, 0].sum() for f in outflows)
    
    print(f"  Cumulative inflow: {total_cumulative_inflow:.2f} Mg")
    print(f"  Cumulative outflow: {total_cumulative_outflow:.2f} Mg")
    
    # FIXED: Add safety check for division by zero
    if total_cumulative_inflow > 0:
        ratio = total_cumulative_outflow / total_cumulative_inflow
        print(f"  Ratio (out/in): {ratio:.3f}")
        
        if ratio > 1.1:  # Allow small numerical errors
            print(f"  ❌ CRITICAL: Outflow exceeds total input by {(ratio-1)*100:.1f}%!")
        elif ratio > 0.95:
            print(f"  ✅ Good: Outflow ratio is reasonable")
        else:
            print(f"  ⚠️ Low outflow ratio - check if DSM is working correctly")
    else:
        print(f"  ❌ Zero inflow detected!")
    
    # Enhanced diagnostics
    params = DSM_PARAMS[process_id]
    lifetimes = params['lifetimes']['Mean']
    std_devs = params['lifetimes']['StdDev']
    
    print(f"  DSM Configuration:")
    print(f"    Lifetimes: {lifetimes} years")
    print(f"    StdDevs: {std_devs} years")
    
    # Check for problematic lifetimes
    for i, (lt, std) in enumerate(zip(lifetimes, std_devs)):
        if lt < 1:
            print(f"    ❌ Category {i}: Lifetime {lt} < 1 year causes immediate outflows")
        elif std > lt:
            print(f"    ⚠️ Category {i}: StdDev {std} > Mean {lt} causes unrealistic distribution")
        else:
            print(f"    ✅ Category {i}: Parameters look reasonable")
    
    # Check annual flows for the last few years
    print(f"  Recent annual flows (last 5 years):")
    for year_idx in range(-5, 0):
        year = mfa_system_with_results.IndexTable.Classification['Time'].Items[year_idx]
        annual_inflow = sum(f.Values[year_idx, 0] for f in inflows)
        annual_outflow = sum(f.Values[year_idx, 0] for f in outflows)
        print(f"    {year}: In={annual_inflow:.2f}, Out={annual_outflow:.2f}")

=== DSM OUTFLOW VERIFICATION ===

Process 2:
  Cumulative inflow: 4779.74 Mg
  Cumulative outflow: 6536.83 Mg
  Ratio (out/in): 1.368
  ❌ CRITICAL: Outflow exceeds total input by 36.8%!
  DSM Configuration:
    Lifetimes: [6] years
    StdDevs: [2] years
    ✅ Category 0: Parameters look reasonable
  Recent annual flows (last 5 years):
    2041: In=4.48, Out=8.32
    2042: In=0.00, Out=8.50
    2043: In=18.06, Out=8.99
    2044: In=18.06, Out=9.54
    2045: In=4.48, Out=9.79

Process 3:
  Cumulative inflow: 4761.56 Mg
  Cumulative outflow: 4092.94 Mg
  Ratio (out/in): 0.860
  ⚠️ Low outflow ratio - check if DSM is working correctly
  DSM Configuration:
    Lifetimes: [80] years
    StdDevs: [16] years
    ✅ Category 0: Parameters look reasonable
  Recent annual flows (last 5 years):
    2041: In=7.11, Out=77.19
    2042: In=7.24, Out=77.01
    2043: In=7.63, Out=76.87
    2044: In=8.06, Out=76.79
    2045: In=8.26, Out=76.75


In [29]:
# Analyze flow magnitudes for geographic realism
print("\n=== FLOW MAGNITUDE ANALYSIS ===")
primary_flow = mfa_system_with_results.FlowDict.get('F_00_01')
if primary_flow:
    total_input_2045 = primary_flow.Values[-1, 0]  # 2045, material
    cumulative_input = primary_flow.Values[:, 0].sum()
    
    print(f"Annual timber input (2045): {total_input_2045:.2f} Mg")
    print(f"Cumulative input (1920-2045): {cumulative_input:.2f} Mg")
    
    # Reality check for neighborhood scale
    # Eichkamp is a small Berlin neighborhood
    print("\nReality check for Eichkamp Kiez:")
    print(f"  Average per year: {cumulative_input/126:.2f} Mg")
    
    # Rough estimate: ~1000-2000 housing units in typical Berlin kiez
    # Each unit might use 0.1-1 Mg timber per year for maintenance/renovation
    estimated_range_low = 1000 * 0.1  # 100 Mg/year
    estimated_range_high = 2000 * 1.0  # 2000 Mg/year
    
    if total_input_2045 < estimated_range_low:
        print(f"    ⚠️ Input seems LOW for neighborhood scale")
    elif total_input_2045 > estimated_range_high:
        print(f"    ⚠️ Input seems HIGH for neighborhood scale")
    else:
        print(f"    ✅ Input magnitude seems reasonable")


=== FLOW MAGNITUDE ANALYSIS ===
Annual timber input (2045): 6.40 Mg
Cumulative input (1920-2045): 6828.20 Mg

Reality check for Eichkamp Kiez:
  Average per year: 54.19 Mg
    ⚠️ Input seems LOW for neighborhood scale


In [30]:
# Analyze temporal trends
print("\n=== TEMPORAL TRENDS ANALYSIS ===")
years = mfa_system_with_results.IndexTable.Classification['Time'].Items

# Look at primary input trends
primary_flow = mfa_system_with_results.FlowDict.get('F_00_01')
if primary_flow:
    early_avg = primary_flow.Values[:25, 0].mean()  # 1920-1945
    recent_avg = primary_flow.Values[-25:, 0].mean()  # 2020-2045
    
    print(f"Early period average (1920-1945): {early_avg:.2f} Mg/year")
    print(f"Recent period average (2020-2045): {recent_avg:.2f} Mg/year")
    print(f"Growth factor: {recent_avg/early_avg:.1f}x")
    
    # Check for realistic urbanization/construction trends
    if recent_avg > early_avg * 10:
        print("    ⚠️ Very high growth - check if realistic for neighborhood")
    elif recent_avg < early_avg:
        print("    ⚠️ Declining trend - unusual for urban development")
    else:
        print("    ✅ Growth trend seems reasonable")


=== TEMPORAL TRENDS ANALYSIS ===
Early period average (1920-1945): 84.65 Mg/year
Recent period average (2020-2045): 11.54 Mg/year
Growth factor: 0.1x
    ⚠️ Declining trend - unusual for urban development


In [31]:
# Check DSM outflow timing
print("\n=== DSM TIMING ANALYSIS ===")
for process_id in DSM_PARAMS.keys():
    outflows = [f for f in mfa_system_with_results.FlowDict.values() if f.P_Start == process_id]
    if outflows:
        outflow = outflows[0]  # Take first outflow
        
        # Find when significant outflows start
        cumulative_outflow = np.cumsum(outflow.Values[:, 0])
        first_significant = np.where(cumulative_outflow > cumulative_outflow[-1] * 0.01)[0]
        
        if len(first_significant) > 0:
            delay_years = first_significant[0]
            mean_lifetime = DSM_PARAMS[process_id]['lifetimes']['Mean'][0]
            
            print(f"Process {process_id}:")
            print(f"  Expected lifetime: {mean_lifetime} years")
            print(f"  First outflows after: {delay_years} years")
            
            if delay_years > mean_lifetime * 2:
                print(f"    ⚠️ Outflows delayed too long vs. expected lifetime")
            else:
                print(f"    ✅ Outflow timing consistent with lifetime")


=== DSM TIMING ANALYSIS ===
Process 0:
  Expected lifetime: 20 years
  First outflows after: 0 years
    ✅ Outflow timing consistent with lifetime
Process 2:
  Expected lifetime: 6 years
  First outflows after: 5 years
    ✅ Outflow timing consistent with lifetime
Process 3:
  Expected lifetime: 80 years
  First outflows after: 57 years
    ✅ Outflow timing consistent with lifetime
Process 7:
  Expected lifetime: 70 years
  First outflows after: 100 years
    ✅ Outflow timing consistent with lifetime


In [32]:
# DEBUG cell before running MFA calculation
print("=== TC PARAMETER MATCHING DEBUG ===")

# Check what TC parameters actually exist
available_tcs = [name for name in mfa_system_with_results.ParameterDict.keys() if name.startswith('TC_')]
print(f"Available TC parameters: {available_tcs}")

# Check what the code expects for each flow
expected_missing_pairs = []
for flow_name, flow_obj in mfa_system_with_results.FlowDict.items():
    if flow_name != 'F_00_01':  # Skip primary input
        expected_tc = f"TC_{'_'.join(flow_name.split('_')[1:3])}"
        exists = expected_tc in mfa_system_with_results.ParameterDict
        
        print(f"Flow {flow_name} expects TC: {expected_tc} - {'✅ EXISTS' if exists else '❌ MISSING'}")
        
        if not exists:
            expected_missing_pairs.append((flow_name, expected_tc))

print(f"\n🔍 MISSING TC ANALYSIS:")
print(f"Found {len(expected_missing_pairs)} flows without matching TCs")

# Check your Excel TC sheet structure
if 'all_excel_data' in globals():
    tc_sheet = all_excel_data.get('2_5_dynamic_tcs')
    if tc_sheet is not None:
        print(f"\nTC Excel data preview:")
        print(tc_sheet.head())
        print(f"Available TC_IDs in Excel: {list(tc_sheet['TC_ID'].unique())}")

=== TC PARAMETER MATCHING DEBUG ===
Available TC parameters: ['TC_07_00', 'TC_00_01', 'TC_05_00', 'TC_05_07', 'TC_06_07', 'TC_06_08', 'TC_02_03', 'TC_02_08', 'TC_01_00', 'TC_01_02', 'TC_04_03', 'TC_04_08', 'TC_03_04', 'TC_03_05', 'TC_03_06']
Flow F_01_02 expects TC: TC_01_02 - ✅ EXISTS
Flow F_01_00 expects TC: TC_01_00 - ✅ EXISTS
Flow F_02_03 expects TC: TC_02_03 - ✅ EXISTS
Flow F_03_04 expects TC: TC_03_04 - ✅ EXISTS
Flow F_04_03 expects TC: TC_04_03 - ✅ EXISTS
Flow F_03_05 expects TC: TC_03_05 - ✅ EXISTS
Flow F_03_06 expects TC: TC_03_06 - ✅ EXISTS
Flow F_05_07 expects TC: TC_05_07 - ✅ EXISTS
Flow F_05_00 expects TC: TC_05_00 - ✅ EXISTS
Flow F_06_07 expects TC: TC_06_07 - ✅ EXISTS
Flow F_06_08 expects TC: TC_06_08 - ✅ EXISTS
Flow F_02_08 expects TC: TC_02_08 - ✅ EXISTS
Flow F_04_08 expects TC: TC_04_08 - ✅ EXISTS
Flow F_07_00 expects TC: TC_07_00 - ✅ EXISTS

🔍 MISSING TC ANALYSIS:
Found 0 flows without matching TCs


In [33]:
# Add this to your MFA script instead of AppMap
def visualize_mfa_system():
    """Visualize MFA system architecture without AppMap"""
    print("""
    🗺️ MFA SYSTEM ARCHITECTURE MAP
    ===============================
    
    📊 EXECUTION FLOW:
    ┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────┐
    │   Excel Data    │───▶│  System Setup       │───▶│ MFA Calculation │
    │   (6 sheets)    │    │  (Processes/Flows)  │    │  (Enhanced)     │
    └─────────────────┘    └──────────────────────┘    └─────────────────┘
             │                        │                         │
             ▼                        ▼                         ▼
    ┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────┐
    │ Data Validation │    │ Dynamic Parameters   │    │ Results & DSM   │
    │ & Processing    │    │ (Time Series TCs)    │    │ Integration     │
    └─────────────────┘    └──────────────────────┘    └─────────────────┘
    
    🔧 MAIN COMPONENTS:
    ├── 📁 Input Layer: Excel sheets (2_1 to 2_6)
    ├── 🏗️ Object Layer: 8 Processes, 15 Flows, 5 Stocks  
    ├── 🔄 Parameter Layer: 15 Dynamic TCs (126 time steps)
    ├── 🧮 Calculation Layer: Enhanced MFA solver + DSM
    └── 📊 Output Layer: Flow series + Stock evolution
    
    ⏱️ TIME DIMENSION: 1920-2045 (126 years)
    🧪 ELEMENTS: CC, DM, WC (multi-element tracking)
    🔄 DYNAMICS: Time-varying transfer coefficients
    """)

def trace_function_calls():
    """Manual function call tracing"""
    call_sequence = [
        "1. 📂 load_and_define_processes()",
        "   ├── pd.read_excel() - Load 6 Excel sheets",
        "   ├── Create 8 Process objects", 
        "   ├── Create 15 Flow objects",
        "   └── Initialize 5 Stock objects",
        "",
        "2. 🔄 create_dynamic_transfer_coefficients()",
        "   ├── Extract TC data from Excel",
        "   ├── np.interp() - Interpolate 126 time steps",
        "   ├── Create 15 dynamic Parameter objects",
        "   └── Normalize for mass balance",
        "",
        "3. 🧮 run_mfa_calculation()",
        "   ├── Resolve calculation dependencies",
        "   ├── Iterative flow calculation (3 rounds)",
        "   ├── DSM integration for 5 stocks",
        "   └── Mass balance verification",
        "",
        "4. 📊 Results processing & analysis"
    ]
    
    print("=== FUNCTION CALL TRACE ===")
    for step in call_sequence:
        print(step)

# Use these instead of AppMap
visualize_mfa_system()
trace_function_calls()


    🗺️ MFA SYSTEM ARCHITECTURE MAP
    
    📊 EXECUTION FLOW:
    ┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────┐
    │   Excel Data    │───▶│  System Setup       │───▶│ MFA Calculation │
    │   (6 sheets)    │    │  (Processes/Flows)  │    │  (Enhanced)     │
    └─────────────────┘    └──────────────────────┘    └─────────────────┘
             │                        │                         │
             ▼                        ▼                         ▼
    ┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────┐
    │ Data Validation │    │ Dynamic Parameters   │    │ Results & DSM   │
    │ & Processing    │    │ (Time Series TCs)    │    │ Integration     │
    └─────────────────┘    └──────────────────────┘    └─────────────────┘
    
    🔧 MAIN COMPONENTS:
    ├── 📁 Input Layer: Excel sheets (2_1 to 2_6)
    ├── 🏗️ Object Layer: 8 Processes, 15 Flows, 5 Stocks  
    ├── 🔄 Parameter Layer: 15 Dynamic TCs (126 time steps)
    ├── 🧮 Ca

# Section 3: Main workflow (execution)

In [34]:

# ====================================================================
# Section 3: Main Execution / Workflow (Corrected Version)
# ====================================================================

# --- 1. Define Model Scope ---
# We call our new function and pass the parameters from the configuration block
model_classification, index_table = define_model_scope(START_YEAR, END_YEAR, ELEMENTS)

# --- 2. Initialize the MFA System Object ---
# The output of the first function is the input for this one
my_mfa_system = initialize_mfa_system(model_classification, index_table)

#visualize_mfa_skeleton(my_mfa_system)

# --- 3. Load Data and Define Processes/Stocks ---
# The mfa_system object is passed in and gets modified by the function.
# We also receive all excel data to use it in the next steps.
my_mfa_system, all_excel_data = load_and_define_processes(my_mfa_system, EXCEL_FILE_PATH)

# --- 4. Define Flows and Parameters ---
# This function completes the setup of the system object.
# This is the correct call that unpacks the tuple.
mfa_system_with_results, _ = define_flows_and_parameters(my_mfa_system, all_excel_data, DSM_PARAMS)

mfa_system_with_results, dsm_details = run_mfa_calculation(mfa_system_with_results)

# --- 5. Calculation Phase ---
print("\nRunning MFA calculations...")

print("Calculation complete.")

# --- 6. Display final mass balance (optional) ---
balance = mfa_system_with_results.MassBalance()
print("\nFinal Mass Balance Check (Sum of absolute errors per process):")
print(np.abs(balance).sum(axis=0).sum(axis=1))
plot_mass_balance_error(mfa_system_with_results)

print("\n✅ System setup is complete and ready for calculations!")



--> Model scope and classifications defined.
--> MFA system object initialized.
--> Excel file '250731_V11.xlsx' loaded successfully.

=== PROCESSING PROCESS DEFINITIONS ===
  Processing: Biosphere (ID: 0)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_0, S_0
  Processing: Raw Material Extraction (ID: 1)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Processing (ID: 2)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_2, S_2
  Processing: Use (ID: 3)
    TC: True, Dynamic TC: True, Stock: True
    ✅ Created stocks: dS_3, S_3
  Processing: Refurbishment (ID: 4)
    TC: True, Dynamic TC: True, Stock: False
  Processing: Degredation (ID: 5)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Incineration (ID: 6)
    TC: True, Dynamic TC: False, Stock: False
  Processing: Atmosphere (ID: 7)
    TC: True, Dynamic TC: False, Stock: True
    ✅ Created stocks: dS_7, S_7
  Processing: Anthroposphere (ID: 8)
    TC: True, Dynamic TC:

interactive(children=(IntSlider(value=1920, description='Year', max=2045, min=1920), Dropdown(description='Ele…

FigureWidget({
    'data': [{'marker': {'color': [#2ca02c, #7f7f7f, #2ca02c, #2ca02c, #7f7f7f,
                                   #7f7f7f, #7f7f7f, #d62728, #d62728]},
              'type': 'bar',
              'uid': '0f19fb53-24e6-4960-ac03-4f1e17f12162',
              'x': [Biosphere, Raw Material Extraction, Processing, Use,
                    Refurbishment, Degredation, Incineration, Atmosphere,
                    Anthroposphere],
              'y': [-302.5999739767458, 0.0, -0.1429677005299652,
                    -8.811111956719131e-08, 0.0, 0.0, 0.0, 7.100116961367142e-08,
                    0.14296777911277508]}],
    'layout': {'shapes': [{'line': {'color': 'black', 'width': 2},
                           'type': 'line',
                           'x0': -0.5,
                           'x1': 8.5,
                           'y0': 0,
                           'y1': 0}],
               'template': '...',
               'title': {'text': 'Mass Balance Error Check for MATERIAL


✅ System setup is complete and ready for calculations!


### Überprüfung der Massenbilanz

Die folgende Grafik ist das wichtigste Werkzeug zur Überprüfung der Modellkonsistenz. Sie zeigt den Massenbilanzfehler für jeden Prozess, berechnet nach der Formel:

**`Fehler = Σ Zuflüsse - Σ Abflüsse - Lagerveränderung (dS)`**

**Wie man die Grafik liest:**
* **Perfektes Gleichgewicht:** Ein Prozess ist perfekt bilanziert, wenn sein Balken genau auf der Nulllinie liegt.
* **Positiver Fehler (Balken > 0):** Es wurde mehr Masse "erschaffen" als im System sein dürfte. Mögliche Ursache: Ein Abfluss oder eine Lagerbildung fehlt in der Definition.
* **Negativer Fehler (Balken < 0):** Es ist Masse "verschwunden". Mögliche Ursache: Ein Zufluss fehlt oder ein Abfluss wird doppelt gezählt.

Je größer die Abweichung von Null, desto gravierender ist der Fehler in der Modelllogik für diesen Prozess.

# Section 4: Results and visualization

In [35]:
def plot_interactive_sankey(mfa_system_results):
    import plotly.graph_objects as go
    from ipywidgets import interact, IntSlider, Dropdown, SelectMultiple, FloatSlider

    # --- NEW: Build process_id_to_name including system boundary ---
    process_id_to_name = {p.ID: p.Name for p in mfa_system_results.ProcessList}
    if 0 not in process_id_to_name:
        process_id_to_name[0] = "System"  # or "Outside"

    all_process_names = list(process_id_to_name.values())
    all_flows = list(mfa_system_results.FlowDict.values())
    time_items = mfa_system_results.IndexTable.Classification['Time'].Items
    element_items = mfa_system_results.Elements
    max_flow_value = max(f.Values.max() for f in all_flows if f.Values is not None) if all_flows else 1

    fig = go.FigureWidget(data=[go.Sankey(node=dict(label=[]), link=dict(source=[], target=[], value=[]))])

    def update_sankey(year, element, processes_to_show, min_flow_value):
        if not processes_to_show:
            with fig.batch_update(): fig.data[0].node.label = []
            return

        # --- REPLACE THIS BLOCK WITH THE FOLLOWING ---
        label_map = {pid: i for i, pid in enumerate(process_id_to_name.keys()) if process_id_to_name[pid] in processes_to_show}
        filtered_labels = [name for name in processes_to_show]

        year_index = time_items.index(year)
        element_index = element_items.index(element)

        candidate_flows = [f for f in all_flows if f.P_Start in label_map and f.P_End in label_map]
        final_flows = [f for f in candidate_flows if f.Values[year_index, element_index] >= min_flow_value]

        with fig.batch_update():
            if not final_flows:
                fig.data[0].node.label = filtered_labels
                fig.data[0].link.source, fig.data[0].link.target, fig.data[0].link.value = [], [], []
            else:
                fig.data[0].node.label = filtered_labels
                fig.data[0].node.color = "blue"
                fig.data[0].link.source = [label_map[f.P_Start] for f in final_flows]
                fig.data[0].link.target = [label_map[f.P_End] for f in final_flows]
                fig.data[0].link.value = [f.Values[year_index, element_index] for f in final_flows]
            fig.update_layout(title_text=f"MFA Sankey for {element.upper()} in {year} (Flows > {min_flow_value:.2f} Mg)",
                              font_size=20, height=700, margin=dict(l=10, r=10, b=20, t=50))

    # --- REPLACE THIS BLOCK WITH THE FOLLOWING ---
    process_selector = SelectMultiple(options=all_process_names, value=list(all_process_names), description='Processes', rows=8)
    year_slider = IntSlider(min=time_items[0], max=time_items[-1], step=1, value=time_items[0], description='Year')
    element_dropdown = Dropdown(options=element_items, value=element_items[0], description='Element')
    threshold_slider = FloatSlider(min=0, max=max_flow_value, step=max_flow_value/100, value=0,
                                   description='Min Flow', continuous_update=False, readout_format='.2f')

    interact(update_sankey, year=year_slider, element=element_dropdown, processes_to_show=process_selector, min_flow_value=threshold_slider)
    display(fig)

In [36]:
# --- Helper Function 4.3: Plot Inflow, Stock, and Outflow (Final Corrected Version) ---
def plot_process_dynamics(mfa_system_results, process_definitions):
    """
    Creates three side-by-side line charts showing the dynamics of 
    Inflow, Stock, and Outflow, using process type metadata for smarter titles.
    """
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    from ipywidgets import interact, Dropdown

    # <<< HIER IST DIE ANPASSUNG: Der korrekte Spaltenname aus deiner Excel-Datei >>>
    PROCESS_TYPE_COLUMN_NAME = 'Process_Type' 

    # Check if the column exists to avoid errors
    has_type_column = PROCESS_TYPE_COLUMN_NAME in process_definitions.columns
    if not has_type_column:
        print(f"Warning: Column '{PROCESS_TYPE_COLUMN_NAME}' not found in '2_1_Definition_Processes'. Smart titles will be disabled.")

    process_options = {p.Name: p.ID for p in mfa_system_results.ProcessList if f"S_{p.ID}" in mfa_system_results.StockDict}
    if not process_options:
        print("No processes with stocks found to plot.")
        return
        
    element_items = mfa_system_results.Elements
    time_axis = mfa_system_results.IndexTable.Classification['Time'].Items
    
    fig = go.FigureWidget(make_subplots(rows=1, cols=3, subplot_titles=("Inflow", "Stock (S)", "Outflow")))

    def update_plot(process_name, element):
        pid = process_options[process_name]
        element_index = element_items.index(element)

        inflows = [f.Values[:, element_index] for f in mfa_system_results.FlowDict.values() if f.P_End == pid]
        inflow_ts = sum(inflows) if inflows else np.zeros(len(time_axis))
        stock_ts = mfa_system_results.StockDict[f'S_{pid}'].Values[:, element_index]
        outflows = [f.Values[:, element_index] for f in mfa_system_results.FlowDict.values() if f.P_Start == pid]
        outflow_ts = sum(outflows) if outflows else np.zeros(len(time_axis))
        
        subplot_titles = (f"Inflow to '{process_name}'", f"Stock in '{process_name}'", f"Outflow from '{process_name}'")
        
        if has_type_column:
            process_type = process_definitions.loc[process_definitions['ID'] == pid, PROCESS_TYPE_COLUMN_NAME].iloc[0]
            if process_type == 'Input':
                subplot_titles = ("Primary System Input", f"Stock in '{process_name}'", f"Outflow from '{process_name}'")
            elif process_type == 'Output':
                subplot_titles = (f"Inflow to '{process_name}'", f"Stock in '{process_name}'", "Final System Output (Sink)")

        with fig.batch_update():
            fig.data, fig.layout.annotations = [], []
            fig.add_trace(go.Scatter(x=time_axis, y=inflow_ts, mode='lines', name='Inflow'), row=1, col=1)
            fig.add_trace(go.Scatter(x=time_axis, y=stock_ts, mode='lines', name='Stock'), row=1, col=2)
            fig.add_trace(go.Scatter(x=time_axis, y=outflow_ts, mode='lines', name='Outflow'), row=1, col=3)
            
            fig.layout.annotations = [
                dict(x=0.155, y=1.05, text=subplot_titles[0], showarrow=False, xref='paper', yref='paper', xanchor='center'),
                dict(x=0.5, y=1.05, text=subplot_titles[1], showarrow=False, xref='paper', yref='paper', xanchor='center'),
                dict(x=0.845, y=1.05, text=subplot_titles[2], showarrow=False, xref='paper', yref='paper', xanchor='center')
            ]
            fig.update_layout(title=f"Dynamics for Process: '{process_name}' | Element: {element.upper()}", height=400, showlegend=False)
            fig.update_xaxes(title_text="Year")
            fig.update_yaxes(title_text="Mass in Mg", row=1, col=1)

    process_dropdown = Dropdown(options=list(process_options.keys()), description='Process:')
    element_dropdown = Dropdown(options=element_items, value=element_items[0], description='Element:')
    
    interact(update_plot, process_name=process_dropdown, element=element_dropdown)
    display(fig)

In [37]:

# --- Helper Function 4.4: Plot dynamic stock composition from pre-calculated results ---
def plot_dynamic_stock_composition(dsm_details, mfa_system_results):
    """
    Visualizes the pre-calculated stock composition from the dsm_details dictionary.
    """
    import plotly.graph_objects as go
    from ipywidgets import interact, Dropdown, Checkbox

    # Create dropdown options from the available results
    process_options = list(dsm_details.keys())
    if not process_options:
        print("DSM calculation was not run or produced no results. Nothing to plot.")
        return
        
    element_items = mfa_system_results.Elements
    time_axis = mfa_system_results.IndexTable.Classification['Time'].Items
    
    fig = go.FigureWidget()

    def update_plot(process_id, element, show_as_bars):
        details = dsm_details.get(process_id, {})
        stock_by_category = details.get('stock_by_category', [])
        category_labels = details.get('category_names', [])
        
        # Find the inflow flow to get the elemental content parameter
        inflow_flow = next((f for f in mfa_system_results.FlowDict.values() if f.P_End == process_id), None)
        
        # Determine the multiplication factor for the selected element
        content_factor = 1.0
        if element != 'material' and inflow_flow:
            param_name = f"{element}_{inflow_flow.Name}"
            if param_name in mfa_system_results.ParameterDict:
                content_factor = mfa_system_results.ParameterDict[param_name].Values
            else: # If no parameter is found, the elemental stock is zero
                content_factor = 0.0

        with fig.batch_update():
            fig.data = [] # Clear previous traces
            chart_type = go.Bar if show_as_bars else go.Scatter
            
            for i, material_stock_ts in enumerate(stock_by_category):
                # Calculate the elemental stock by applying the content factor
                element_stock_ts = material_stock_ts * content_factor
                
                trace_props = dict(x=time_axis, y=element_stock_ts, name=category_labels[i], hoverinfo='x+y')
                if not show_as_bars:
                    trace_props.update(mode='lines', line=dict(width=0.5), stackgroup='one')
                fig.add_trace(chart_type(**trace_props))

            process_name = next((p.Name for p in mfa_system_results.ProcessList if p.ID == process_id), "")
            fig.update_layout(
                barmode='stack' if show_as_bars else None,
                title=f"Dynamic Stock Composition for Process: '{process_name}' ({element.upper()})",
                xaxis_title="Year", yaxis_title=f"Stock in Mg"
            )

    interact(update_plot, 
             process_id=Dropdown(options=process_options, description='Process:'), 
             element=Dropdown(options=element_items, value=element_items[0], description='Element:'),
             show_as_bars=Checkbox(value=False, description='Show as Bar Chart'))
    display(fig)


In [38]:
# # --- Helper Function 4.5: Plot the dynamics of a FOMP process ---
# def plot_fomp_dynamics(mfa_system_results, FOMP_PARAMS):
#     """
#     Creates side-by-side line charts for Inflow, Stock, and Outflow
#     for a process calculated with FOMP. Includes interactive widgets.
#     """
#     from plotly.subplots import make_subplots
#     import plotly.graph_objects as go
#     from ipywidgets import interact, Dropdown

#     # Create a mapping of process names to IDs for the dropdown, only for FOMP processes
#     process_options = {
#         p.Name: p.ID 
#         for p in mfa_system_results.ProcessList 
#         if p.ID in FOMP_PARAMS
#     }
#     if not process_options:
#         print("No processes with FOMP parameters are defined in the configuration.")
#         return
        
#     element_items = mfa_system_results.Elements
#     time_axis = mfa_system_results.IndexTable.Classification['Time'].Items
    
#     fig = go.FigureWidget(make_subplots(rows=1, cols=3, subplot_titles=("Total Inflow", "Absolute Stock (S)", "Outflow (Mineralization)")))

#     def update_plot(process_name, element):
#         pid = process_options[process_name]
#         element_index = element_items.index(element)

#         # Get the time series data for the selected process
#         inflow_ts = sum(f.Values[:, element_index] for f in mfa_system_results.FlowDict.values() if f.P_End == pid)
#         stock_ts = mfa_system_results.StockDict.get(f'S_{pid}').Values[:, element_index]
#         outflow_ts = sum(f.Values[:, element_index] for f in mfa_system_results.FlowDict.values() if f.P_Start == pid)
        
#         with fig.batch_update():
#             fig.data = [] # Clear existing data
#             fig.add_trace(go.Scatter(x=time_axis, y=inflow_ts, mode='lines', name='Inflow'), row=1, col=1)
#             fig.add_trace(go.Scatter(x=time_axis, y=stock_ts, mode='lines', name='Stock'), row=1, col=2)
#             fig.add_trace(go.Scatter(x=time_axis, y=outflow_ts, mode='lines', name='Outflow'), row=1, col=3)
            
#             title_text = f"FOMP Dynamics for Process: '{process_name}' | Element: {element.upper()}"
#             fig.update_layout(title_text=title_text, height=400, showlegend=False)
#             fig.update_xaxes(title_text="Year")
#             fig.update_yaxes(title_text="Mass [Mg]", row=1, col=1)

#     # Create widgets for interaction
#     process_dropdown = Dropdown(options=list(process_options.keys()), description='Process:')
#     element_dropdown = Dropdown(options=element_items, value=element_items[0], description='Element:')
    
#     interact(update_plot, process_name=process_dropdown, element=element_dropdown)
#     display(fig)

In [39]:
# --- Helper Function 4.6: Plot the dynamics of selected flows over time ---
def plot_flow_dynamics(mfa_system_results):
    """
    Creates an interactive line/bar chart to show the development of selected
    flows over time for a chosen element.
    """
    import plotly.graph_objects as go
    from ipywidgets import interact, Dropdown, SelectMultiple, Checkbox

    # Create options for the widgets
    flow_options = sorted(list(mfa_system_results.FlowDict.keys()))
    if not flow_options:
        print("No flows found in the system to plot.")
        return
        
    element_items = mfa_system_results.Elements
    time_axis = mfa_system_results.IndexTable.Classification['Time'].Items
    
    
    # Use FigureWidget for efficient updates
    fig = go.FigureWidget()

    def update_plot(flows_to_show, element, show_as_bars):
        # Use batch_update for smooth interaction
        with fig.batch_update():
            fig.data = [] # Clear previous traces
            if not flows_to_show:
                fig.update_layout(title_text="Please select one or more flows to display.")
                return

            element_index = element_items.index(element)
            chart_type = go.Bar if show_as_bars else go.Scatter

            # Add a trace for each selected flow
            for flow_id in flows_to_show:
                flow_obj = mfa_system_results.FlowDict.get(flow_id)
                if flow_obj:
                    trace_props = dict(x=time_axis, y=flow_obj.Values[:, element_index], name=flow_id)
                    if not show_as_bars:
                        trace_props.update(mode='lines')
                    fig.add_trace(chart_type(**trace_props))
            
            # Update layout and title
            fig.update_layout(
                barmode='stack' if show_as_bars else 'overlay',
                title=f"Time Series for Selected Flows ({element.upper()})",
                xaxis_title="Year",
                yaxis_title="Mass in Mg",
                hovermode="x unified"
            )

    # Create widgets
    flow_selector = SelectMultiple(options=flow_options, value=[flow_options[0]] if flow_options else [], description='Flows:', rows=10)
    element_dropdown = Dropdown(options=element_items, value=element_items[0], description='Element:')
    chart_type_checkbox = Checkbox(value=False, description='Show as Bar Chart')

    interact(update_plot, flows_to_show=flow_selector, element=element_dropdown, show_as_bars=chart_type_checkbox)
    display(fig)

In [40]:
print("Flows:", list(mfa_system_with_results.FlowDict.keys()))
print("Processes:", [p.Name for p in mfa_system_with_results.ProcessList])

print("Processes loaded:")
for p in mfa_system_with_results.ProcessList:
    print(f"ID: {p.ID}, Name: {p.Name}")

print("\nFlows loaded:")
for f in mfa_system_with_results.FlowDict.values():
    print(f"{f.Name}: {f.P_Start} -> {f.P_End}")

for f in mfa_system_with_results.FlowDict.values():
    print(f"Flow data: {f.Name}: {f.P_Start} -> {f.P_End}, max={f.Values.max()}, sum={f.Values.sum()}")

Flows: ['F_00_01', 'F_01_02', 'F_01_00', 'F_02_03', 'F_03_04', 'F_04_03', 'F_03_05', 'F_03_06', 'F_05_07', 'F_05_00', 'F_06_07', 'F_06_08', 'F_02_08', 'F_04_08', 'F_07_00']
Processes: ['Biosphere', 'Raw Material Extraction', 'Processing', 'Use', 'Refurbishment', 'Degredation', 'Incineration', 'Atmosphere', 'Anthroposphere']
Processes loaded:
ID: 0, Name: Biosphere
ID: 1, Name: Raw Material Extraction
ID: 2, Name: Processing
ID: 3, Name: Use
ID: 4, Name: Refurbishment
ID: 5, Name: Degredation
ID: 6, Name: Incineration
ID: 7, Name: Atmosphere
ID: 8, Name: Anthroposphere

Flows loaded:
F_00_01: 0 -> 1
F_01_02: 1 -> 2
F_01_00: 1 -> 0
F_02_03: 2 -> 3
F_03_04: 3 -> 4
F_04_03: 4 -> 3
F_03_05: 3 -> 5
F_03_06: 3 -> 6
F_05_07: 5 -> 7
F_05_00: 5 -> 0
F_06_07: 6 -> 7
F_06_08: 6 -> 8
F_02_08: 2 -> 8
F_04_08: 4 -> 8
F_07_00: 7 -> 0
Flow data: F_00_01: 0 -> 1, max=302.6, sum=17070.5
Flow data: F_01_02: 1 -> 2, max=211.82, sum=11949.349999999999
Flow data: F_01_00: 1 -> 0, max=90.78, sum=5121.15
Flow 

In [41]:
# #mark all processes widget

# all_process_names = [p.Name for p in mfa_system_with_results.ProcessList]

# process_selector = SelectMultiple(
#     options=all_process_names,
#     value=list(all_process_names),  # <-- ensures all are selected by default
#     description='Processes',
#     rows=8
# )

In [42]:
# dealing with flow 0
process_id_to_name = {p.ID: p.Name for p in mfa_system_with_results.ProcessList}
if 0 not in process_id_to_name:
    process_id_to_name[0] = "System"

all_process_names = list(process_id_to_name.values())


In [43]:
# ====================================================================
# Section 4: Results & Visualization (Final Version)
# ====================================================================

# --- 4.1 Interactive Sankey Diagram ---
print("Displaying interactive Sankey diagram:")
plot_interactive_sankey(mfa_system_with_results)


# --- 4.2 Process Dynamics Plot (Inflow-Stock-Outflow) ---
print("\nDisplaying Inflow-Stock-Outflow Dynamics:")
plot_process_dynamics(mfa_system_with_results, all_excel_data['2_1_Definition_Processes'])

# --- 4.3 Dynamic Stock Composition Plot ---
print("\nDisplaying Dynamic Stock Composition:")
if dsm_details:
    plot_dynamic_stock_composition(dsm_details, mfa_system_with_results)
else:
    print("--> DSM calculation was not run or no DSM processes are defined. Nothing to display.")

# # --- 4.4 FOMP Dynamics Plot ---
# print("\nDisplaying FOMP Dynamics:")
# if FOMP_PARAMS and any(f in mfa_system_with_results.StockDict for f in [f"S_{pid}" for pid in FOMP_PARAMS]):
#     plot_fomp_dynamics(mfa_system_with_results, FOMP_PARAMS)
# else:
#     print("--> No FOMP parameters defined or no FOMP stocks present. Nothing to display.")

# --- 4.5 Flow Dynamics Plot ---
print("\nDisplaying Flow Dynamics:")
plot_flow_dynamics(mfa_system_with_results)

Displaying interactive Sankey diagram:


interactive(children=(IntSlider(value=1920, description='Year', max=2045, min=1920), Dropdown(description='Ele…

FigureWidget({
    'data': [{'link': {'source': [0, 1, 1, 2, 3, 4, 3, 3, 5, 5, 6, 6, 2, 4, 7],
                       'target': [1, 2, 0, 3, 4, 3, 5, 6, 7, 0, 7, 8, 8, 8, 0],
                       'value': [302.6, 211.82, 90.78, 0.285935401059902,
                                 8.196383216585446e-08, 4.098191608292723e-09,
                                 2.0490958041463614e-08, 7.171835314512265e-08,
                                 1.9466410139390434e-08, 1.0245479020731807e-09,
                                 7.100116961367142e-08, 7.171835314512265e-10,
                                 0.142967700529951, 7.786564055756174e-08, 0.0]},
              'node': {'color': 'blue',
                       'label': [Biosphere, Raw Material Extraction, Processing,
                                 Use, Refurbishment, Degredation, Incineration,
                                 Atmosphere, Anthroposphere]},
              'type': 'sankey',
              'uid': '181cd5e6-3799-4dc1-9079-35f72847


Displaying Inflow-Stock-Outflow Dynamics:


interactive(children=(Dropdown(description='Process:', options=('Biosphere', 'Processing', 'Use', 'Atmosphere'…

...

interactive(children=(Dropdown(description='Process:', options=(0, 2, 3, 7, 8), value=0), Dropdown(description…

FigureWidget({
    'data': [{'hoverinfo': 'x+y',
              'line': {'width': 0.5},
              'mode': 'lines',
              'name': 'Average Exchange (20 yrs)',
              'stackgroup': 'one',
              'type': 'scatter',
              'uid': 'c5488889-a053-4031-a8ac-861c82a8de1e',
              'x': [1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929,
                    1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
                    1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949,
                    1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959,
                    1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
                    1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
                    1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
                    1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
                    2000, 2001, 2002, 2003, 2004, 2005, 2006, 


Displaying Flow Dynamics:


interactive(children=(SelectMultiple(description='Flows:', index=(0,), options=('F_00_01', 'F_01_00', 'F_01_02…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'F_00_01',
              'type': 'scatter',
              'uid': '7fc7149f-7a3e-4fac-970a-930d9a5ddb0d',
              'x': [1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929,
                    1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
                    1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949,
                    1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959,
                    1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
                    1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
                    1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
                    1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
                    2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
                    2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
                    2020, 2

# Section 5: Export Results

In [44]:
# ====================================================================
# Section 5: Export Results
# ====================================================================

def export_results_to_excel(mfa_system_results, output_filename="mfa_results.xlsx"):
    """
    Exports all calculated flows and stocks into a single Excel file with multiple sheets.
    """
    print(f"\n--> Exporting results to '{output_filename}'...")
    
    time_index = mfa_system_results.IndexTable.Classification['Time'].Items
    elements = mfa_system_results.Elements
    
    with pd.ExcelWriter(output_filename) as writer:
        # --- Export Flows ---
        flow_data_rows = []
        for name, flow_obj in mfa_system_results.FlowDict.items():
            for i, year in enumerate(time_index):
                row = {'Flow_ID': name, 'Year': year}
                for j, element in enumerate(elements):
                    row[element] = flow_obj.Values[i, j]
                flow_data_rows.append(row)
        df_flows = pd.DataFrame(flow_data_rows)
        df_flows.to_excel(writer, sheet_name='Flows_ts', index=False)
        
        # --- Export Stocks ---
        stock_data_rows = []
        for name, stock_obj in mfa_system_results.StockDict.items():
            for i, year in enumerate(time_index):
                row = {'Stock_ID': name, 'Year': year}
                for j, element in enumerate(elements):
                    row[element] = stock_obj.Values[i, j]
                stock_data_rows.append(row)
        df_stocks = pd.DataFrame(stock_data_rows)
        df_stocks.to_excel(writer, sheet_name='Stocks_ts', index=False)
        
    print("--> Export complete.")

In [45]:
# ====================================================================
# Section 5: Export Results
# ====================================================================

# Call the export function
export_results_to_excel(mfa_system_with_results, output_filename="timber_thruss_results.xlsx")


--> Exporting results to 'timber_thruss_results.xlsx'...
--> Export complete.


<img src="system_flow_diagram.svg" alt="system_flow_diagram">

## 0 Load packages

This cell imports all necessary packages. It also loads the ODYM framework and the bioDYM_addon, both are included as files in the project (see folder /framework). 

## 3 MFA Calculations

Now, the solution of the MFA is calculated. Since most flows have either input data or TCs and substance contents are given, they can be easily calculated. However, this system includes a dynamic stock modeling (dsm) and a first order model process (FOMP) for the mineralization of carbon in soil. The idea is that first, all flows are calculated with TCs that are independent of dsm or FOMP. Then, dsm is performed and subsequently, all flows up to the FOMP are calculated. After that, the bioDYM_addon functions are used to calculate the mineralization. Finally, all following flows and stocks are calculated.

### 3.1 Solution MFA

### 3.1.1 Solution MFA pt. I (until MBC dynamic stock modelling)


### 3.1.4 Solution MFA pt. IV (FOMP mineralization process)

The carbon mineralization process in the soil is calculated with a first order decay model according to (Cayuela et al., 2010) based on (Robertson & Paul, 2000): 

$$ C_{remaining} (t)=f \cdot exp⁡(-k_{1} \cdot t)+(100\%-f) \cdot exp⁡(-k_{2} \cdot t) $$

To keep calculations simple, it is assumed that this is the only equation that leads to outflows of the process, the remaining fractions of the material accumulate as stock without any emissions. (Cayuela et al., 2010) give parameter values for green waste biochar, they are used here.

\
\
Literature

Cayuela, M. L., Oenema, O., Kuikman, P. J., Bakker, R. R., & Van GROENIGEN, J. W. (2010). Bioenergy by-products as soil amendments? Implications for carbon sequestration and greenhouse gas emissions: C AND N DYNAMICS FROM BIOENERGY BY-PRODUCTS IN SOIL. GCB Bioenergy, no-no. https://doi.org/10.1111/j.1757-1707.2010.01055.x

Robertson, G. P., & Paul, E. (2000). Decomposition and Soil Organic Matter Dynamics. Decomposition and Soil Organic Matter Dynamics., 104–116. https://doi.org/10.1007/978-1-4612-1224-9_8